<a href="https://www.kaggle.com/code/ryancardwell/rhodiumorcav3?scriptVersionId=271837991" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Cell 1 — Safe Imports, Config, DebugContext, PerformanceMetrics (Merged & Safe for v3)
import os, json, math, time, random
from collections import Counter, defaultdict
from dataclasses import dataclass
from functools import lru_cache
from typing import Any, Dict, List, Tuple, Callable, Optional
import numpy as np
import itertools
from pathlib import Path
import warnings
import datetime
from scipy import ndimage, spatial, signal
import networkx as nx

warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# ---- SEEDING (idempotent; merged with original) ----
def set_seed(seed: int = 42):
    try:
        import torch  # optional
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    except Exception:
        pass
    random.seed(seed)
    np.random.seed(seed)

set_seed(42)  # Ensures reproducibility; safe to call multiple times

# ---- CONFIG (non-destructive; only set if missing; merged with original) ----
if 'GRID_DTYPE' not in globals():
    GRID_DTYPE = np.int8
if 'TIME_LIMIT_PER_TASK' not in globals():
    TIME_LIMIT_PER_TASK = 30.0  # seconds
if 'TOTAL_TIME_BUDGET' not in globals():
    TOTAL_TIME_BUDGET = 7200.0  # seconds
MAX_GRID_DIM = 30  # Retained from original
MIN_GRID_DIM = 1   # Retained from original

# Basic debug context (define if missing; from patch)
try:
    DebugContext
except NameError:
    class DebugContext:
        def __init__(self, time_limit_s: float = TIME_LIMIT_PER_TASK):
            self.t0 = time.time()
            self.time_limit_s = time_limit_s
            self.pathways = []
        def check_time(self) -> bool:
            return (time.time() - self.t0) < self.time_limit_s
        def log_pathway(self, name: str, payload: Any, score: float = 0.0):
            self.pathways.append((name, payload, score))
        def elapsed(self) -> float:
            return time.time() - self.t0

# IMPL 8: Performance tracking (safe define/extend from patch; merged with original)
try:
    PerformanceMetrics
    # Extend with extra helper if not present
    if not hasattr(PerformanceMetrics, 'log_failure_reason'):
        def _log_failure_reason(self, task_id: str, reason: str):
            entry = {"task_id": task_id, "reason": reason, "ts": time.time()}
            if not hasattr(self, "_fail_reasons"):
                self._fail_reasons = []
            self._fail_reasons.append(entry)
        PerformanceMetrics.log_failure_reason = _log_failure_reason
except NameError:
    class PerformanceMetrics:
        def __init__(self):
            self.success = 0
            self.fail = 0
            self.records = []
            self._fail_reasons = []
            self.metrics = defaultdict(list)
            self.start_time = time.time()
            self.task_count = 0
            self.success_count = 0
            self.failure_reasons = Counter()
        
        def log_metric(self, metric_name: str, value: Any):
            self.metrics[metric_name].append((time.time(), value))
        
        def log_success(self, task_id: str, method: str, confidence: float, time_taken: float):
            self.success_count += 1
            self.log_metric("successful_tasks", {
                "task_id": task_id,
                "method": method,
                "confidence": confidence,
                "time_taken": time_taken
            })
        
        def log_failure(self, task_id: str, reason: str, time_taken: float):
            self.failure_reasons[reason] += 1
            self.log_metric("failed_tasks", {
                "task_id": task_id,
                "reason": reason,
                "time_taken": time_taken
            })
        
        def log_failure_reason(self, task_id: str, reason: str):
            self._fail_reasons.append({"task_id": task_id, "reason": reason, "ts": time.time()})
        
        def summary(self) -> Dict[str, Any]:
            tot = self.success + self.fail
            return {
                "total": tot,
                "success": self.success,
                "fail": self.fail,
                "avg_time": float(np.mean([r["dt"] for r in self.records])) if self.records else 0.0,
                "slowest_top3": sorted(self.records, key=lambda r: r["dt"], reverse=True)[:3],
                "fail_reasons": self._fail_reasons,
            }
        
        def save_metrics(self):
            metrics_file = Path('/kaggle/working/performance_metrics.json')
            summary = {
                "system": "RhodiumOrca-v3",
                "timestamp": datetime.datetime.now().isoformat(),
                "total_tasks": self.task_count,
                "successful_tasks": self.success_count,
                "success_rate": self.success_count / max(1, self.task_count),
                "total_runtime": time.time() - self.start_time,
                "failure_breakdown": dict(self.failure_reasons),
                "detailed_metrics": dict(self.metrics)
            }
            with open(metrics_file, 'w') as f:
                json.dump(summary, f, indent=2, default=str)
            print(f"📊 Performance metrics saved to: {metrics_file}")

# Global metrics instance (retained from original)
metrics = PerformanceMetrics()

# IMPL 9: Early validation constants (retained from original)
MIN_TRAIN_EXAMPLES = 1
MAX_TRAIN_EXAMPLES = 10

print(f"🚀 RhodiumOrca v3 - Metacognitive R&D Solver (Seed: 42)")
print("=" * 80)
print("🔧 Configuration:")
print(f"   - Grid dtype: {GRID_DTYPE}")
print(f"   - Time limit per task: {TIME_LIMIT_PER_TASK}s")
print(f"   - Total budget: {TOTAL_TIME_BUDGET/3600:.1f}h")
print(f"   - Grid dimensions: {MIN_GRID_DIM}-{MAX_GRID_DIM}")
print(f"   - Train examples: {MIN_TRAIN_EXAMPLES}-{MAX_TRAIN_EXAMPLES}")
print("=" * 80)
print("Cell 1 ready: imports, seed, config, DebugContext, PerformanceMetrics (merged & safe).")

🚀 RhodiumOrca v3 - Metacognitive R&D Solver (Seed: 42)
🔧 Configuration:
   - Grid dtype: <class 'numpy.int8'>
   - Time limit per task: 30.0s
   - Total budget: 2.0h
   - Grid dimensions: 1-30
   - Train examples: 1-10
Cell 1 ready: imports, seed, config, DebugContext, PerformanceMetrics (merged & safe).


In [2]:
# Cell 2 — Core Data Utilities and Grid Wrapper (v3 Dataclass, V2 Utilities)

# Patch: Re-import essential dependencies required for this cell, in case Cell 1 was skipped or failed.
from dataclasses import dataclass, field
from functools import lru_cache
# FIX: Import the required type hints 'Set' and others
from typing import Dict, List, Any, Tuple, Optional, Callable, Set, Deque 
import numpy as np

# ---- IMPL 2: Core Grid Wrapper (v3 Dataclass, V2 Utilities as Methods) ----

@dataclass(frozen=True)
class Grid:
    """
    A feature-rich wrapper for the core ARC grid (np.ndarray).
    V3 Enhancement: Immutable (frozen=True) for cache-key stability.
    """
    data: np.ndarray = field(hash=True, compare=True)
    
    # V3 Safety: Check for ARC color constraints (0-9)
    def __post_init__(self):
        if self.data.dtype != GRID_DTYPE:
             # V3 Robustness: Convert to the standard DTYPE if needed
             object.__setattr__(self, 'data', self.data.astype(GRID_DTYPE))
             
        if self.data.ndim != 2:
            warnings.warn("Grid initialized with non-2D data.", UserWarning)

    # --- Basic Properties (Cached for efficiency - V2/V3 Core) ---
    
    @property
    @lru_cache(maxsize=1)
    def shape(self) -> Tuple[int, int]:
        return self.data.shape

    @property
    @lru_cache(maxsize=1)
    def rows(self) -> int:
        return self.data.shape[0] if self.data.ndim == 2 else 0

    @property
    @lru_cache(maxsize=1)
    def cols(self) -> int:
        return self.data.shape[1] if self.data.ndim == 2 else 0

    @property
    @lru_cache(maxsize=1)
    def size(self) -> int:
        return self.data.size

    @property
    @lru_cache(maxsize=1)
    def is_valid(self) -> bool:
        return self.data.ndim == 2 and self.data.size > 0

    @property
    @lru_cache(maxsize=1)
    def unique_colors(self) -> Set[int]:
        """Returns the set of unique colors present in the grid."""
        if self.data.size == 0:
            return set()
        return set(self.data.flatten())

    # --- V2 Geometric Transformations (Methods for immutability) ---

    @lru_cache(maxsize=1)
    def rotate_90(self, k: int = 1) -> 'Grid':
        """V2 Feature: Rotates the grid 90 degrees clockwise k times."""
        if not self.is_valid: return self
        rotated_data = np.rot90(self.data, k=-k) # k=-k for clockwise rotation logic
        return Grid(rotated_data)

    @lru_cache(maxsize=1)
    def flip_h(self) -> 'Grid':
        """V2 Feature: Flips the grid horizontally."""
        if not self.is_valid: return self
        return Grid(np.fliplr(self.data))

    @lru_cache(maxsize=1)
    def flip_v(self) -> 'Grid':
        """V2 Feature: Flips the grid vertically."""
        if not self.is_valid: return self
        return Grid(np.flipud(self.data))
        
    @lru_cache(maxsize=1)
    def all_symmetries(self) -> List['Grid']:
        """
        V2 Advanced Tech: Returns all 8 rotational/reflectional symmetries.
        Includes identity, 3 rotations, H-flip, V-flip, and their rotations.
        """
        if not self.is_valid: return [self]
        symmetries = []
        current = self
        
        # 4 Rotations
        for _ in range(4):
            symmetries.append(current)
            current = current.rotate_90()
            
        # 4 Flips/Rotations of the horizontal flip
        flipped = self.flip_h()
        for _ in range(4):
            symmetries.append(flipped)
            flipped = flipped.rotate_90()
            
        # V3 Robustness: Use a set comprehension to ensure only unique grids are returned
        return list({s: s for s in symmetries}.keys())

    @lru_cache(maxsize=1)
    def crop_to_bbox(self) -> 'Grid':
        """
        V2 Feature: Crops the grid to the bounding box of non-zero pixels.
        Returns a Grid of the smallest possible size containing all objects.
        """
        if not self.is_valid: return self
        
        non_zero_indices = np.argwhere(self.data != 0)
        if non_zero_indices.size == 0:
            return Grid(np.array([], dtype=GRID_DTYPE).reshape((0, 0)))

        r_min, c_min = non_zero_indices.min(axis=0)
        r_max, c_max = non_zero_indices.max(axis=0)
        
        # Slicing is inclusive of the start and exclusive of the end (hence r_max + 1)
        cropped_data = self.data[r_min:r_max+1, c_min:c_max+1]
        
        return Grid(cropped_data)

    def get_bbox(self) -> Tuple[int, int, int, int]:
        """Returns (r_min, r_max, c_min, c_max) of non-zero pixels."""
        if not self.is_valid: return (0, 0, 0, 0)
        non_zero_indices = np.argwhere(self.data != 0)
        if non_zero_indices.size == 0:
            return (0, 0, 0, 0)
        r_min, c_min = non_zero_indices.min(axis=0)
        r_max, c_max = non_zero_indices.max(axis=0)
        return (r_min, r_max + 1, c_min, c_max + 1)


# ---- IMPL 3: Data Structures (V2/V3 Task Definition) ----

@dataclass(frozen=True)
class Example:
    """Represents a single input/output pair in a task."""
    input: Grid
    output: Grid
    
@dataclass(frozen=True)
class Task:
    """Represents a full ARC task."""
    task_id: str
    train: List[Example]
    test: List[Example]

@dataclass(frozen=True)
class Object:
    """
    V3 Core: Represents a single connected component (object) detected in a grid.
    Encapsulates position, shape, and content (itself a Grid).
    """
    grid: Grid                 # The cropped content of the object
    position: Tuple[int, int]  # (r_min, c_min) of the object in the original grid
    color: int                 # The dominant/first color detected (V2 heuristic)
    shape: Tuple[int, int]     # (rows, cols) of the object's bounding box
    count: int                 # Total number of pixels in the object

# ---- IMPL 3: Flattener and Unflattener (V2 Competition Requirement) ----

def flattener(grid_data: np.ndarray) -> str:
    """
    V2 Competition Requirement: Converts a 2D array into the required output string format.
    Example: [[1,1], [0,2]] -> "1102"
    """
    # Flattens the array and joins the integer representations
    return "".join(map(str, grid_data.flatten()))

def unflatten_output(flat_string: str, rows: int, cols: int) -> Grid:
    """
    V2 Utility: Converts the flattened string back to a Grid (used for testing/debugging).
    """
    if len(flat_string) != rows * cols:
        warnings.warn(f"Flattened string length ({len(flat_string)}) does not match required size ({rows*cols}).", UserWarning)
        # Attempt to pad/truncate safely
        flat_list = [int(c) for c in flat_string if '0' <= c <= '9']
        data = np.array(flat_list).reshape((rows, cols))
        return Grid(data)
    
    data = np.array([int(c) for c in flat_string]).reshape((rows, cols))
    return Grid(data)

# NOTE: The actual task loading (load_task, get_all_task_ids) is assumed to be in Cell 3.
# We ensure the constants are defined here for the type hints to work.
if 'GRID_DTYPE' not in globals():
    GRID_DTYPE = np.int8 # Ensure GRID_DTYPE is available

print("Cell 2 ready: Core Grid Wrapper (Grid), Data Structures (Task, Example, Object), and I/O Utilities (flattener, unflatten_output) loaded.")


Cell 2 ready: Core Grid Wrapper (Grid), Data Structures (Task, Example, Object), and I/O Utilities (flattener, unflatten_output) loaded.


In [3]:
# Cell 2 — Core Grid Utilities (v3 Framework + v2 Features)

# V2 Feature: Basic Grid Validation, enforced to use v3's configuration
def is_valid_grid(grid: Any) -> bool:
    """
    Checks if the input is a valid 2D NumPy array for ARC tasks:
    - Must be a 2D array.
    - Must use the configured GRID_DTYPE (np.int8).
    - Dimensions must be between MIN_GRID_DIM (1) and MAX_GRID_DIM (30).
    - All values must be in the valid color range [0, 9].
    """
    if not isinstance(grid, np.ndarray):
        return False
    if grid.ndim != 2:
        return False
    # Use the global configuration from Cell 1
    if grid.dtype != GRID_DTYPE:
        return False
        
    rows, cols = grid.shape
    
    # ARC constraints: MIN_GRID_DIM <= dim <= MAX_GRID_DIM
    if not (MIN_GRID_DIM <= rows <= MAX_GRID_DIM and MIN_GRID_DIM <= cols <= MAX_GRID_DIM):
        return False
        
    # V2 Feature: Color range check [0, 9]
    if grid.size > 0 and (np.min(grid) < 0 or np.max(grid) > 9):
        return False
    return True

# V2 Feature: Essential Conversions (using v3's GRID_DTYPE)
def array_to_list(array: np.ndarray) -> List[List[int]]:
    """Convert a 2D numpy array to a list of lists (required for flattener)."""
    return array.tolist()

def list_to_array(lst: List[List[int]]) -> np.ndarray:
    """Convert a list of lists to a 2D numpy array with GRID_DTYPE."""
    # Uses the global configuration from Cell 1
    return np.array(lst, dtype=GRID_DTYPE)

# V2 Feature: String representation for internal state/debugging
def grid_to_s(grid: np.ndarray) -> str:
    """Compact string representation of a grid."""
    if grid.size == 0:
        return ""
    # Use array_to_list for a clean conversion to string characters
    return '\n'.join(''.join(map(str, row)) for row in array_to_list(grid))

def s_to_grid(s: str) -> np.ndarray:
    """Convert a compact string representation back to a grid."""
    lines = [line.strip() for line in s.strip().split('\n') if line.strip()]
    if not lines:
        # Returns an empty 0x0 array if the input string is empty
        return np.array([], dtype=GRID_DTYPE).reshape((0, 0))
    
    lst = [[int(c) for c in line] for line in lines]
    return list_to_array(lst)


# V2 Feature: Official ARC Submission Flattener (Core Feature)
def flattener(grid: np.ndarray) -> str:
    """
    Converts a single predicted output grid into the official ARC submission string format.
    Format: "R<rows>C<cols>:<r1c1...r1cn>;...;<rmc1...rmcn>"
    Returns an empty string for invalid/empty grids (V3 Safety).
    """
    # V3 Safety: Check validity before submission
    if not is_valid_grid(grid) or grid.size == 0:
        return ""
    
    rows, cols = grid.shape
    grid_list = array_to_list(grid)
    
    # R<rows>C<cols>:
    s = f"R{rows}C{cols}:"
    
    # <r1c1...r1cn>;...;<rmc1...rmcn>
    s += ";".join("".join(map(str, row)) for row in grid_list)
    
    return s

print("Cell 2 ready: Core Grid Utilities (v2/v3 Merged) loaded.")


Cell 2 ready: Core Grid Utilities (v2/v3 Merged) loaded.


In [4]:
# Cell 3 — Dataset Loading and Task Parsing (v2 Utility + v3 Path Robustness)

# IMPL 3: Data Loading Utilities

# Patch: Ensure core imports and dependencies are available
from dataclasses import dataclass, field
from typing import Dict, List, Any, Tuple, Optional
import numpy as np
import json
import os
from pathlib import Path
import warnings
import time

# NOTE: Grid, Example, and Task dataclasses (from Cell 2) are assumed to be available.

# --- V3 Path Configuration ---
# Standard ARC data paths (Assumes running in a Kaggle-like environment)
DATA_PATH = Path('/kaggle/input/abstraction-and-reasoning-corpus/')
TRAINING_PATH = DATA_PATH / 'training'
EVALUATION_PATH = DATA_PATH / 'evaluation'
TESTING_PATH = DATA_PATH / 'test' # The actual final test set, used identically to evaluation

# --- Core Utility ---

def load_json_data(file_path: Path) -> Optional[Dict[str, Any]]:
    """V3 Safety: Reads a JSON file, handling errors."""
    if not file_path.exists():
        warnings.warn(f"File not found: {file_path}", UserWarning)
        return None
    try:
        with open(file_path, 'r') as f:
            return json.load(f)
    except Exception as e:
        warnings.warn(f"Error loading JSON {file_path}: {e}", UserWarning)
        return None

def parse_grid_data(raw_data: List[List[int]]) -> Grid:
    """V2 Utility: Converts a raw list-of-lists into a Grid object."""
    if not raw_data:
        return Grid(np.array([], dtype=GRID_DTYPE).reshape((0, 0)))
    
    # V3 Robustness: Use list comprehension for speed and convert to standard DTYPE
    data = np.array(raw_data, dtype=GRID_DTYPE)
    return Grid(data)

# --- IMPL 3: Primary Data Loaders ---

def get_all_task_ids() -> List[str]:
    """
    V3 Core: Scans the training and evaluation directories to find all task IDs.
    This function populates the ALL_TASK_IDS global list.
    """
    all_task_ids: List[str] = []
    
    # Check for both training and evaluation tasks
    for path in [TRAINING_PATH, EVALUATION_PATH, TESTING_PATH]:
        if path.exists():
            for file_path in path.glob('*.json'):
                # Task ID is the filename without the extension
                task_id = file_path.stem
                if task_id not in all_task_ids:
                    all_task_ids.append(task_id)

    if not all_task_ids:
        warnings.warn("No ARC task JSON files found in standard data paths. Solver will target 0 tasks.", UserWarning)
        
    return all_task_ids

def load_task(task_id: str) -> Optional[Task]:
    """
    V2/V3 Core: Loads a single task by ID from the appropriate directory (Train or Eval).
    """
    
    # V3 Robustness: Check all known task directories
    file_path = None
    for path in [TRAINING_PATH, EVALUATION_PATH, TESTING_PATH]:
        p = path / f'{task_id}.json'
        if p.exists():
            file_path = p
            break
            
    if file_path is None:
        warnings.warn(f"Task ID {task_id} not found in any standard directory.", UserWarning)
        return None

    raw_task_data = load_json_data(file_path)
    if raw_task_data is None:
        return None

    train_examples: List[Example] = []
    test_examples: List[Example] = []

    # V3 Safety: Check structure
    if 'train' in raw_task_data:
        for ex in raw_task_data['train']:
            input_grid = parse_grid_data(ex['input'])
            output_grid = parse_grid_data(ex['output'])
            train_examples.append(Example(input=input_grid, output=output_grid))
            
    if 'test' in raw_task_data:
        for ex in raw_task_data['test']:
            input_grid = parse_grid_data(ex['input'])
            # Output might be missing in test/evaluation set, use a dummy 0x0 grid if necessary
            output_data = ex.get('output', []) 
            output_grid = parse_grid_data(output_data) 
            test_examples.append(Example(input=input_grid, output=output_grid))
    
    # V3 Consistency: Ensure task always has at least one example
    if not train_examples and not test_examples:
        warnings.warn(f"Task {task_id} contains no valid examples.", UserWarning)
        return None

    return Task(task_id=task_id, train=train_examples, test=test_examples)

# --- Global Initialization Block ---
# This block is essential for setting up the execution context for Cell 10/12.

try:
    # Set the global variable that run_solver (Cell 10) depends on
    ALL_TASK_IDS = get_all_task_ids()
    print(f"✅ Data Load Success: Found {len(ALL_TASK_IDS)} task IDs.")
except NameError:
    # If a dependency (like Grid or GRID_DTYPE) failed in Cell 2, fall back safely
    print("⚠️ WARNING: Data loading failed due to missing dependencies. ALL_TASK_IDS set to empty list.")
    ALL_TASK_IDS = []

print("Cell 3 ready: Data Loading and Task Parsing utilities loaded.")


✅ Data Load Success: Found 0 task IDs.
Cell 3 ready: Data Loading and Task Parsing utilities loaded.


In [5]:
# Cell 4 — The Feature-Rich Grid Object (v3 Dataclass + v2 Utilities)

# IMPL 2: Core Grid Wrapper (v3 Dataclass, V2 Utilities as Methods)
@dataclass(frozen=True)
class Grid:
    """
    A feature-rich wrapper for the core ARC grid (np.ndarray).
    Uses V3's dataclass with frozen=True to ensure grids are immutable and hashable,
    enabling aggressive caching (lru_cache on methods).
    """
    data: np.ndarray

    def __post_init__(self):
        # V3 Safety: Enforce configuration and validity check from Cell 2
        if not is_valid_grid(self.data):
             # Use object.__setattr__ to bypass frozen=True for controlled error handling
             object.__setattr__(self, 'is_valid', False)
             warnings.warn("Grid initialized with invalid data! Operations may fail.", UserWarning)
        else:
             object.__setattr__(self, 'is_valid', True)
    
    # ---- Core Properties (V3/V2 Accessors) ----

    @property
    @lru_cache(maxsize=1)
    def shape(self) -> Tuple[int, int]:
        """Returns the (rows, cols) of the grid."""
        return self.data.shape

    @property
    @lru_cache(maxsize=1)
    def rows(self) -> int:
        """Returns the number of rows."""
        return self.data.shape[0]

    @property
    @lru_cache(maxsize=1)
    def cols(self) -> int:
        """Returns the number of columns."""
        return self.data.shape[1]

    @property
    @lru_cache(maxsize=1)
    def size(self) -> int:
        """Returns the total number of cells."""
        return self.data.size

    @property
    @lru_cache(maxsize=1)
    def unique_colors(self) -> Set[int]:
        """Returns the set of unique colors present in the grid."""
        if self.data.size == 0:
            return set()
        return set(np.unique(self.data).tolist())

    # ---- Geometric Transformations (V2 Core Features) ----

    @lru_cache(maxsize=32)
    def rotate_90(self, k: int = 1) -> 'Grid':
        """
        Rotates the grid 90 degrees clockwise k times.
        V2: Implements full geometric rotation.
        V3: Uses lru_cache for performance.
        """
        if k % 4 == 0:
            return self
        new_data = np.rot90(self.data, k=-k) # numpy's k is counter-clockwise, use -k for clockwise
        return Grid(new_data.astype(GRID_DTYPE))

    @lru_cache(maxsize=4)
    def flip_h(self) -> 'Grid':
        """Flips the grid horizontally (left-right)."""
        new_data = np.fliplr(self.data)
        return Grid(new_data.astype(GRID_DTYPE))

    @lru_cache(maxsize=4)
    def flip_v(self) -> 'Grid':
        """Flips the grid vertically (up-down)."""
        new_data = np.flipud(self.data)
        return Grid(new_data.astype(GRID_DTYPE))

    def all_symmetries(self) -> List['Grid']:
        """
        Returns all 8 possible symmetric versions of the grid (rotation and reflection).
        V2: Essential for matching and hypothesis testing.
        """
        symmetries = []
        current_grid = self
        
        # 4 Rotations
        for k in range(4):
            rotated = current_grid.rotate_90(k=k)
            symmetries.append(rotated)
            
            # 4 Rotations of the horizontal flip
            flipped = rotated.flip_h()
            symmetries.append(flipped)

        # Use a set of compact string representations to ensure uniqueness
        unique_grids = {}
        for grid in symmetries:
            if grid.is_valid:
                unique_grids[grid.to_s()] = grid
        
        return list(unique_grids.values())

    # ---- Structural Manipulation (V2/V3 Utilities) ----

    @lru_cache(maxsize=16)
    def get_bbox(self, target_color: Optional[int] = None) -> Tuple[int, int, int, int]:
        """
        Calculates the minimum bounding box (r_min, r_max, c_min, c_max) 
        of all non-background pixels, or for a specific color.
        V2: The most fundamental feature for object-centric operations.
        Returns: (r_start, r_end+1, c_start, c_end+1) - Python slicing format
        """
        if self.size == 0:
            return (0, 0, 0, 0)
        
        if target_color is not None:
            # Mask for the specific color
            mask = (self.data == target_color)
        else:
            # Mask for all non-background (non-black/color 0) pixels
            mask = (self.data != 0)
        
        if not np.any(mask):
            # If no pixels match, return a null box (0x0 area)
            return (0, 0, 0, 0)
        
        rows, cols = np.where(mask)
        
        r_min, r_max = rows.min(), rows.max()
        c_min, c_max = cols.min(), cols.max()
        
        # Return in slicing format: r_start:r_end, c_start:c_end
        return (r_min, r_max + 1, c_min, c_max + 1)

    @lru_cache(maxsize=16)
    def crop_to_bbox(self, target_color: Optional[int] = None) -> 'Grid':
        """Crops the grid to its minimal bounding box."""
        r_min, r_max, c_min, c_max = self.get_bbox(target_color)
        if r_max == r_min or c_max == c_min:
             # Empty or 0x0 crop area, return an empty grid
             return Grid(np.array([], dtype=GRID_DTYPE).reshape((0, 0)))

        cropped_data = self.data[r_min:r_max, c_min:c_max]
        return Grid(cropped_data.copy())

    @lru_cache(maxsize=16)
    def pad_to_shape(self, target_rows: int, target_cols: int, pad_value: int = 0) -> 'Grid':
        """Pads the grid with 'pad_value' (default 0) to reach the target shape."""
        r, c = self.shape
        if r >= target_rows and c >= target_cols:
            # No padding needed (or target shape is smaller, which we ignore for padding)
            return self
        
        pad_r = max(0, target_rows - r)
        pad_c = max(0, target_cols - c)
        
        # Split padding symmetrically: (before, after)
        pad_r_before = pad_r // 2
        pad_r_after = pad_r - pad_r_before
        pad_c_before = pad_c // 2
        pad_c_after = pad_c - pad_c_before
        
        padding = (
            (pad_r_before, pad_r_after),
            (pad_c_before, pad_c_after)
        )
        
        new_data = np.pad(self.data, padding, mode='constant', constant_values=pad_value)
        return Grid(new_data.astype(GRID_DTYPE))

    # ---- Feature Analysis (V2/V3 Introspection) ----

    @lru_cache(maxsize=2)
    def get_color_counts(self, ignore_color: Optional[int] = 0) -> Counter:
        """
        Counts the occurrence of each color, optionally ignoring one color (default: black/0).
        V2: Essential for frequency-based reasoning.
        """
        if self.data.size == 0:
            return Counter()
        
        # Flatten and count
        counts = Counter(self.data.flatten())
        
        if ignore_color is not None and ignore_color in counts:
            del counts[ignore_color]
            
        return counts

    @lru_cache(maxsize=4)
    def get_dominant_color(self, ignore_color: Optional[int] = 0) -> Optional[int]:
        """Returns the most frequent color in the grid."""
        counts = self.get_color_counts(ignore_color=ignore_color)
        if not counts:
            return None
            
        # Get the color with the highest count. `most_common(1)[0]` returns `(color, count)`
        return counts.most_common(1)[0][0]

    # ---- Object and Component Analysis (V2/V3 Advanced Features) ----

    @lru_cache(maxsize=1)
    def connected_components(self, background_color: int = 0) -> List['Grid']:
        """
        Identifies and extracts all non-background connected components (objects) using 
        scipy.ndimage.label. Each object is returned as its own cropped Grid.
        V2: Core of object-centric solvers.
        """
        if self.data.size == 0:
            return []

        # Create a binary mask: True for non-background pixels
        mask = (self.data != background_color)
        
        # Label the connected components
        labeled_array, num_features = ndimage.label(mask, structure=np.ones((3, 3)))
        
        objects = []
        for i in range(1, num_features + 1):
            # Get the slice coordinates for the current component's bounding box
            slices = ndimage.find_objects(labeled_array == i)[0]
            
            # Extract the component's data from the original grid
            component_data = self.data[slices]
            
            # Create a mask for the component to zero out background (pixels *outside* the object)
            component_mask = (labeled_array[slices] == i)
            
            # Apply the mask to isolate the object in its cropped view
            isolated_data = component_data * component_mask
            
            objects.append(Grid(isolated_data.astype(GRID_DTYPE)))
            
        return objects

    # ---- Color & Masking Utilities (V2 Core) ----
    
    @lru_cache(maxsize=16)
    def mask_color(self, target_color: int, new_value: int = 0) -> 'Grid':
        """
        Creates a new grid where all instances of 'target_color' are replaced 
        with 'new_value'. Used for color filtering/recoloring.
        """
        new_data = self.data.copy()
        new_data[new_data == target_color] = new_value
        return Grid(new_data.astype(GRID_DTYPE))

    @lru_cache(maxsize=16)
    def filter_by_color(self, target_color: int) -> 'Grid':
        """
        Extracts only pixels of the target_color, setting all others to 0 (background).
        Equivalent to mask_color(other_color, 0).
        """
        mask = (self.data == target_color)
        new_data = self.data * mask
        return Grid(new_data.astype(GRID_DTYPE))

    # ---- Debugging and Display (V2/V3 Debugging) ----

    def to_s(self) -> str:
        """
        Returns the compact string representation of the grid (using Cell 2 utility).
        V2: Standardized debug output.
        """
        return grid_to_s(self.data)
        
    def __str__(self) -> str:
        """Friendly string representation."""
        return f"Grid({self.rows}x{self.cols}, colors:{len(self.unique_colors)})"

    # ---- Static/Factory Method (V3 Utility) ----

    @staticmethod
    def from_string(s: str) -> 'Grid':
        """
        Creates a Grid instance from a compact string representation (using Cell 2 utility).
        """
        data = s_to_grid(s)
        return Grid(data)

print("Cell 4 ready: Feature-Rich Grid Object (v3 dataclass wrapper for v2 utilities) loaded.")


Cell 4 ready: Feature-Rich Grid Object (v3 dataclass wrapper for v2 utilities) loaded.


In [6]:
# Cell 5 — Pattern Recognition and Feature Extraction Primitives (v2 Full Suite + v3 Caching)

# IMPL 3: Structural Primitives (Operating on Grid objects)

# V3 Feature: The Object Data Structure
@dataclass(frozen=True)
class Object:
    """
    Represents an isolated, connected component/object detected in a Grid.
    V2: Core component of object-centric reasoning modules.
    """
    grid: Grid
    color: int = 0  # The dominant color of the object
    count: int = 0  # Total number of pixels
    position: Tuple[int, int] = (0, 0) # Top-left (row, col) in the original grid's context
    
    # V3 Feature: Caching and fast access
    @property
    @lru_cache(maxsize=1)
    def shape(self) -> Tuple[int, int]:
        return self.grid.shape
    
    @property
    @lru_cache(maxsize=1)
    def dimensions(self) -> Tuple[int, int]:
        return self.grid.rows, self.grid.cols

    def __str__(self) -> str:
        return f"Object(color={self.color}, shape={self.shape[0]}x{self.shape[1]}, pos={self.position}, count={self.count})"

# ---- Core Primitives (Operating on Grid -> Grid) ----

@lru_cache(maxsize=256)
def translate_grid(grid: Grid, dr: int, dc: int, fill_value: int = 0) -> Grid:
    """
    V2 Feature: Translates a grid by (dr, dc), filling new cells with fill_value.
    V3 Feature: Aggressive caching.
    """
    if grid.size == 0:
        return grid
    
    r, c = grid.shape
    new_data = np.full((r, c), fill_value, dtype=GRID_DTYPE)
    
    # Calculate the slicing parameters for the source (src) and destination (dst)
    
    r_start_src, r_end_src = max(0, -dr), min(r, r - dr)
    c_start_src, c_end_src = max(0, -dc), min(c, c - dc)
    
    r_start_dst, r_end_dst = max(0, dr), min(r, r + dr)
    c_start_dst, c_end_dst = max(0, dc), min(c, c + dc)
    
    # Ensure all slices are valid (non-empty)
    if r_start_src >= r_end_src or c_start_src >= c_end_src:
        # Translation moved the entire grid out of bounds
        return Grid(new_data)
        
    if r_start_dst >= r_end_dst or c_start_dst >= c_end_dst:
        # Destination area is empty
        return Grid(new_data)

    # Apply the translation
    new_data[r_start_dst:r_end_dst, c_start_dst:c_end_dst] = \
        grid.data[r_start_src:r_end_src, c_start_src:c_end_src]
        
    return Grid(new_data)


@lru_cache(maxsize=64)
def dilate_grid(grid: Grid, k_size: int = 1, structure: Optional[np.ndarray] = None) -> Grid:
    """
    V2 Feature: Dilates the non-background pixels using a square kernel of size 2*k_size + 1.
    A crucial step for filling gaps or finding potential boundaries.
    """
    if grid.size == 0:
        return grid
        
    # Default structure: square kernel
    if structure is None:
        k = 2 * k_size + 1
        structure = np.ones((k, k), dtype=bool)

    # Use a binary mask (non-zero is True) for dilation
    binary_mask = (grid.data != 0)
    dilated_mask = ndimage.binary_dilation(binary_mask, structure=structure)
    
    # The dilated grid is the original grid where the mask is True, otherwise 0.
    # We use the mask to reintroduce the original colors, but only where dilation occurred.
    # This is a common ARC solver technique: dilation is about expansion, not color change.
    
    # Find the single most dominant non-background color. If multiple, this is a simplification.
    # This is the V2 approach to "simple" dilation color assignment.
    dominant_color = grid.get_dominant_color()
    if dominant_color is None:
        return Grid(np.zeros_like(grid.data, dtype=GRID_DTYPE))

    new_data = np.where(dilated_mask, dominant_color, 0).astype(GRID_DTYPE)
    
    # A cleaner approach often adopted in V3: dilated areas take the color of the nearest original pixel
    # However, for simple primitives, we stick to the dominant color or a simple OR operation.
    # For now, we use a simple OR: original colors are preserved, and new areas are the dominant color.
    final_data = np.where(grid.data != 0, grid.data, new_data)
    
    return Grid(final_data)


@lru_cache(maxsize=64)
def erode_grid(grid: Grid, k_size: int = 1, structure: Optional[np.ndarray] = None) -> Grid:
    """
    V2 Feature: Erodes the non-background pixels, shrinking the objects.
    Useful for removing thin lines or noise.
    """
    if grid.size == 0:
        return grid

    if structure is None:
        k = 2 * k_size + 1
        structure = np.ones((k, k), dtype=bool)

    # Create a binary mask
    binary_mask = (grid.data != 0)
    eroded_mask = ndimage.binary_erosion(binary_mask, structure=structure)

    # The result preserves the original colors *only* where the erosion mask is True
    new_data = grid.data * eroded_mask
    return Grid(new_data.astype(GRID_DTYPE))


# ---- High-Level Feature Extraction (Operating on Grid -> Object/List[Object]) ----

@lru_cache(maxsize=1)
def extract_objects(grid: Grid, background_color: int = 0) -> List[Object]:
    """
    V2 Core Feature: Extracts all connected components and wraps them in Object dataclasses.
    This is the primary method for transitioning from pixel-level to object-level reasoning.
    """
    if grid.size == 0:
        return []

    # 1. Connected components analysis (from Cell 4's Grid method)
    # This returns a list of cropped Grid objects, isolated to the component area.
    isolated_grids = grid.connected_components(background_color)
    
    # 2. Get bounding box coordinates relative to the original grid
    # This requires running the connected component logic again to get the bounding boxes,
    # but we can optimize this by finding the bounding boxes in the *original* context.
    
    mask = (grid.data != background_color)
    labeled_array, num_features = ndimage.label(mask, structure=np.ones((3, 3)))
    
    if num_features == 0:
        return []

    objects: List[Object] = []
    
    for i in range(1, num_features + 1):
        # Find the slice coordinates for the current component in the ORIGINAL grid
        slices = ndimage.find_objects(labeled_array == i)[0]
        r_slice, c_slice = slices
        
        # Determine top-left position (row, col) in the original grid
        r_pos, c_pos = r_slice.start, c_slice.start
        
        # Get the isolated grid from the pre-calculated list (order is consistent)
        comp_grid = isolated_grids[i - 1]
        
        # Get feature metrics
        counts = comp_grid.get_color_counts(ignore_color=0)
        dominant_color = comp_grid.get_dominant_color(ignore_color=0)
        
        # V3 Safety: Handle objects with only background color due to isolation
        if not dominant_color:
            continue
            
        objects.append(Object(
            grid=comp_grid,
            color=dominant_color,
            count=sum(counts.values()), # Total non-background pixels
            position=(r_pos, c_pos)
        ))

    return objects

# ---- Color Mapping Utility (V2 Core) ----

@lru_cache(maxsize=32)
def apply_color_map(grid: Grid, color_map: Dict[int, int], default_color: Optional[int] = None) -> Grid:
    """
    V2 Feature: Applies a direct color-to-color mapping (e.g., {1: 5, 2: 9}).
    Any color not in the map is left unchanged, or set to `default_color` if provided.
    V3 Feature: Uses aggressive caching for repeated maps.
    """
    if not color_map:
        return grid # No map to apply
        
    new_data = grid.data.copy()
    
    # Find colors in the grid that are not in the map
    unmapped_colors = grid.unique_colors - set(color_map.keys())
    
    # Apply the map for colors that *are* in the map
    for old_color, new_color in color_map.items():
        if old_color != new_color:
            new_data[new_data == old_color] = new_color
            
    # Apply default color transformation for unmapped colors
    if default_color is not None:
        for color in unmapped_colors:
            if color != default_color:
                new_data[new_data == color] = default_color
    
    return Grid(new_data.astype(GRID_DTYPE))

# ---- Color Pattern Feature (V2/V3 Simple Symmetry Check) ----

@lru_cache(maxsize=32)
def check_color_symmetry(grid: Grid) -> Dict[str, bool]:
    """
    Checks for simple color-wise vertical/horizontal symmetry, a key V2 heuristic.
    Ignores position, focusing only on the distribution of colors.
    """
    if grid.size == 0:
        return {'horizontal': True, 'vertical': True, 'point': True}

    r, c = grid.shape
    data = grid.data

    # 1. Horizontal Symmetry (Row mirror)
    h_symmetric = True
    for i in range(r // 2):
        if not np.array_equal(data[i, :], data[r - 1 - i, :]):
            h_symmetric = False
            break

    # 2. Vertical Symmetry (Column mirror)
    v_symmetric = True
    for j in range(c // 2):
        if not np.array_equal(data[:, j], data[:, c - 1 - j]):
            v_symmetric = False
            break

    # 3. Point Symmetry (180 degree rotation)
    p_symmetric = np.array_equal(data, np.rot90(data, k=2))
    
    return {
        'horizontal': h_symmetric, 
        'vertical': v_symmetric, 
        'point': p_symmetric
    }


print("Cell 5 ready: Feature Extraction Primitives (Object Dataclass, Translate, Morphological Ops, Color Mapping) loaded.")


Cell 5 ready: Feature Extraction Primitives (Object Dataclass, Translate, Morphological Ops, Color Mapping) loaded.


In [7]:
# Cell 6 — Advanced Grid Analysis and Feature Generation (v2 High-Value Features + v3 Caching)

# IMPL 4: Advanced Analysis Utilities

# ---- Spatial and Geometric Analysis ----

@lru_cache(maxsize=16)
def calculate_centroid(grid: Grid, target_color: Optional[int] = None) -> Tuple[float, float]:
    """
    V2 Feature: Calculates the geometric centroid (center of mass) of the non-background 
    pixels, or of pixels of a specific color.
    Returns: (center_row, center_col)
    """
    if grid.size == 0:
        return (-1.0, -1.0)
        
    data = grid.data
    
    if target_color is not None:
        mask = (data == target_color)
    else:
        # Non-background
        mask = (data != 0)
        
    if not np.any(mask):
        return (-1.0, -1.0)
        
    # Calculate moments using scipy.ndimage for stability and performance
    # This automatically handles the masked data
    center_of_mass = ndimage.center_of_mass(mask.astype(np.int8))
    
    # center_of_mass returns (row, col)
    # V3 Safety: Return 0.0 if center_of_mass fails (which is rare but possible for edge cases)
    if center_of_mass is None:
        return (-1.0, -1.0)
        
    # Use float precision for advanced reasoning
    return center_of_mass[0], center_of_mass[1]


@lru_cache(maxsize=16)
def measure_distance(p1: Tuple[float, float], p2: Tuple[float, float], metric: str = 'manhattan') -> float:
    """
    V2 Feature: Calculates distance between two points (e.g., centroids or object positions).
    Supports 'manhattan' (L1), 'euclidean' (L2), and 'chebyshev' (L-infinity).
    """
    r1, c1 = p1
    r2, c2 = p2
    dr = abs(r1 - r2)
    dc = abs(c1 - c2)
    
    if metric == 'manhattan':
        return dr + dc
    elif metric == 'euclidean':
        return math.sqrt(dr**2 + dc**2)
    elif metric == 'chebyshev':
        return max(dr, dc)
    else:
        warnings.warn(f"Unknown distance metric: {metric}. Defaulting to Euclidean.", UserWarning)
        return math.sqrt(dr**2 + dc**2)

# ---- Advanced Symmetry Analysis (Beyond simple color distribution) ----

@lru_cache(maxsize=4)
def find_reflective_axis(grid: Grid) -> Optional[Tuple[str, float]]:
    """
    V2 Feature: Finds the exact line of reflectional symmetry (horizontal or vertical).
    Returns: ('h' or 'v', coordinate_float) or None.
    e.g., ('h', 4.5) means symmetry across the horizontal line between row 4 and 5.
    e.g., ('v', 3.0) means symmetry across column 3.
    """
    r, c = grid.shape
    data = grid.data
    
    # 1. Horizontal Reflection (Symmetry across a horizontal line)
    
    # Test integer row axes (axis between cells: 0.5, 1.5, ..., r-0.5)
    for i in range(1, r):
        # Mirror is between row i-1 and i. Center coordinate is i - 0.5
        top = data[:i, :]
        bottom = np.flipud(data[i:, :])
        
        # We need the mirrored parts to have the same dimensions to compare
        min_rows = min(top.shape[0], bottom.shape[0])
        
        if min_rows > 0 and np.array_equal(top[-min_rows:, :], bottom[-min_rows:, :]):
            return ('h', i - 0.5)
            
    # Test axis through a cell (axis at row: 1.0, 2.0, ..., r/2) - only possible if r is odd
    if r % 2 == 1:
        i = r // 2
        # Data above row i should mirror data below row i
        top = data[:i, :]
        bottom = np.flipud(data[i+1:, :])
        
        if top.shape[0] == bottom.shape[0] and np.array_equal(top, bottom):
             return ('h', i) # Symmetry line is through row i (zero-indexed)

    # 2. Vertical Reflection (Symmetry across a vertical line)
    
    # Test integer column axes (axis between cells: 0.5, 1.5, ..., c-0.5)
    for j in range(1, c):
        # Mirror is between col j-1 and j. Center coordinate is j - 0.5
        left = data[:, :j]
        right = np.fliplr(data[:, j:])
        
        min_cols = min(left.shape[1], right.shape[1])

        if min_cols > 0 and np.array_equal(left[:, -min_cols:], right[:, -min_cols:]):
            return ('v', j - 0.5)

    # Test axis through a cell (axis at column: 1.0, 2.0, ..., c/2) - only possible if c is odd
    if c % 2 == 1:
        j = c // 2
        # Data left of col j should mirror data right of col j
        left = data[:, :j]
        right = np.fliplr(data[:, j+1:])
        
        if left.shape[1] == right.shape[1] and np.array_equal(left, right):
             return ('v', j) # Symmetry line is through column j

    return None

# ---- Pathfinding Algorithms (A* and BFS - V2 Advanced Tech) ----

def bfs_path(grid: Grid, start: Tuple[int, int], end: Tuple[int, int], traversable_color: Optional[int] = None) -> Optional[List[Tuple[int, int]]]:
    """
    V2 Advanced Tech: Finds the shortest path (BFS) between two points.
    If traversable_color is None, assumes all non-zero cells are traversable.
    Returns: A list of (r, c) tuples representing the path, or None if no path exists.
    """
    if not (0 <= start[0] < grid.rows and 0 <= start[1] < grid.cols and
            0 <= end[0] < grid.rows and 0 <= end[1] < grid.cols):
        return None
        
    data = grid.data
    rows, cols = grid.shape
    
    # 1. Define traversability
    if traversable_color is not None:
        def is_traversable(r, c):
            return data[r, c] == traversable_color
    else:
        # All non-background (0) cells are traversable
        def is_traversable(r, c):
            return data[r, c] != 0

    if not is_traversable(start[0], start[1]) or not is_traversable(end[0], end[1]):
         # Start or end point is not traversable
         return None

    # 2. BFS implementation
    queue = deque([start])
    visited = {start: None} # Stores {current_node: parent_node} for path reconstruction

    while queue:
        r, c = queue.popleft()

        if (r, c) == end:
            # Path found! Reconstruct it.
            path = []
            curr = end
            while curr is not None:
                path.append(curr)
                curr = visited[curr]
            return path[::-1] # Return reversed path (start to end)

        # Explore neighbors (4-connectivity: up, down, left, right)
        for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nr, nc = r + dr, c + dc

            # Check bounds and traversability, and ensure not visited
            if (0 <= nr < rows and 0 <= nc < cols and 
                is_traversable(nr, nc) and 
                (nr, nc) not in visited):
                
                visited[(nr, nc)] = (r, c)
                queue.append((nr, nc))

    return None # No path found

def a_star_path(grid: Grid, start: Tuple[int, int], end: Tuple[int, int], traversable_color: Optional[int] = None) -> Optional[List[Tuple[int, int]]]:
    """
    V2 Advanced Tech: Finds the shortest path using A* search.
    A* is generally overkill for the small ARC grids, but is included for completeness
    and cases where costs might be introduced later (e.g., color-based costs).
    Currently implemented as simple BFS (unit cost), but structure is A* ready.
    """
    # For unit-cost graphs (standard ARC pathfinding), A* degrades to BFS.
    # We maintain the A* function name but call BFS for simplicity and performance,
    # reserving the full A* logic for cost-weighted pathfinding (future V3 extension).
    return bfs_path(grid, start, end, traversable_color)

# ---- Template Matching and Convolution (V2 Core Feature) ----

@lru_cache(maxsize=128)
def convolve_grid(grid: Grid, kernel: np.ndarray) -> np.ndarray:
    """
    V2 Core Feature: Performs 2D convolution. Essential for pattern detection and 
    simple feature extraction (e.g., edge detection, counting local patterns).
    Returns the raw convolved output array (numpy).
    """
    if grid.size == 0 or kernel.size == 0:
        return np.array([], dtype=np.float32)

    # Use signal.convolve2d with 'valid' mode to find only full matches
    # V2 often used 'same' or 'full', but 'valid' is usually best for detection
    # We use 'same' here to return an output of the same size, simplifying coordinate mapping
    return signal.convolve2d(grid.data.astype(np.float32), kernel.astype(np.float32), mode='same', boundary='fill', fillvalue=0.0)

def find_template_matches(grid: Grid, template: Grid, threshold: float = 0.99) -> List[Tuple[int, int]]:
    """
    V2 Feature: Finds locations where a small template grid matches the larger grid.
    Uses normalized cross-correlation (NCC) for robust matching.
    Returns a list of (r, c) tuples, representing the top-left corner of the match.
    """
    if grid.size < template.size or template.size == 0:
        return []
        
    # We rely on OpenCV's method (cv2.matchTemplate) for robust NCC, but since we are limited 
    # to SciPy/NumPy/ndimage, we use a basic correlation approach.
    
    # 1. Prepare data (binary or exact color matching)
    # The V2 standard template match is based on exact color correlation.
    
    # Use template matching function from scipy.ndimage for simplicity
    # Note: ndimage.find_objects is for labeled components, not generic template matching.
    # We fall back to signal.correlate2d which performs cross-correlation.

    result = signal.correlate2d(grid.data, template.data, mode='valid', boundary='fill', fillvalue=0.0)
    
    # Simple correlation score needs normalization or careful interpretation, 
    # but for exact matches where colors are 0-9, the maximum possible correlation 
    # score is sum(template.data ** 2).
    
    # Calculate the max possible score for an exact match (L2 norm squared)
    max_score = np.sum(template.data.astype(np.float32) ** 2)
    
    # Find locations where the correlation score is close to the max possible score
    # We use a threshold based on the maximum possible score
    
    # V2 Robustness Check: If max_score is 0 (template is all 0s), skip.
    if max_score == 0:
        return []

    # Find where the correlation result is very close to a perfect match
    # Use a relative threshold for floating point comparison
    matches = np.argwhere(result > max_score * threshold)
    
    # matches returns coordinates in the result array, which correspond to the top-left 
    # corner of the template in the original grid when using mode='valid'.
    return [tuple(match) for match in matches]


# ---- Topological Features (V2 Graph-based R&D) ----

@lru_cache(maxsize=1)
def grid_to_graph(grid: Grid, background_color: int = 0, connectivity: int = 4) -> nx.Graph:
    """
    V2 R&D Feature: Converts a Grid into a NetworkX Graph.
    Nodes are (row, col) coordinates of non-background pixels.
    Edges connect adjacent (4- or 8-connected) nodes.
    """
    g = nx.Graph()
    rows, cols = grid.shape
    data = grid.data
    
    # Define neighborhood based on connectivity (4-connectivity default)
    neighbors = []
    if connectivity == 4:
        neighbors = [(-1, 0), (1, 0), (0, -1), (0, 1)] # Up, Down, Left, Right
    elif connectivity == 8:
        neighbors = list(itertools.product([-1, 0, 1], [-1, 0, 1]))
        neighbors.remove((0, 0)) # Remove self

    for r in range(rows):
        for c in range(cols):
            if data[r, c] != background_color:
                node = (r, c)
                g.add_node(node, color=int(data[r, c]))
                
                for dr, dc in neighbors:
                    nr, nc = r + dr, c + dc
                    
                    if 0 <= nr < rows and 0 <= nc < cols and data[nr, nc] != background_color:
                        neighbor_node = (nr, nc)
                        # Only add edge if neighbor is already a node (or will be)
                        if (nr > r) or (nr == r and nc > c): # Prevent double-adding edges
                            g.add_edge(node, neighbor_node)

    return g

@lru_cache(maxsize=1)
def extract_graph_features(grid: Grid) -> Dict[str, Any]:
    """
    V2 R&D Feature: Extracts high-level graph metrics for topological analysis.
    Useful for classifying tasks based on structure (e.g., density, connectivity).
    """
    g = grid_to_graph(grid)
    
    if not g.nodes:
        return {
            "num_nodes": 0, "num_edges": 0, "density": 0.0, 
            "is_connected": False, "num_components": 0, "max_clique_size": 0
        }
    
    # 1. Basic metrics
    num_nodes = g.number_of_nodes()
    num_edges = g.number_of_edges()
    
    # 2. Connectivity
    is_connected = nx.is_connected(g) if num_nodes > 0 else True # 0 nodes is technically connected
    
    # 3. Components
    components = list(nx.connected_components(g))
    num_components = len(components)
    
    # 4. Density
    density = nx.density(g)

    # 5. Advanced (expensive, so optional)
    # max_clique_size = nx.graph_clique_number(g) # Too slow for general use
    
    # 6. Center/Periphery (for small graphs)
    try:
        center = nx.center(g)
        periphery = nx.periphery(g)
    except nx.exception.NetworkXNoPath:
        # Happens if graph is not connected
        center = []
        periphery = []
    
    return {
        "num_nodes": num_nodes,
        "num_edges": num_edges,
        "density": density,
        "is_connected": is_connected,
        "num_components": num_components,
        "center_size": len(center),
        "periphery_size": len(periphery),
        "avg_degree": (2 * num_edges) / num_nodes if num_nodes > 0 else 0.0,
        # max_clique_size is omitted for performance
    }


print("Cell 6 ready: Advanced Grid Analysis (Centroid, Symmetry, Pathfinding, Template Matching, Graph Features) loaded.")


Cell 6 ready: Advanced Grid Analysis (Centroid, Symmetry, Pathfinding, Template Matching, Graph Features) loaded.


In [8]:
# Cell 7 — Comparators and Rule Structures (v2 Robustness + v3 Formalization)

# IMPL 5: Comparison and Difference Analysis

# Patch: Ensure required imports are available
from dataclasses import dataclass, field
from functools import lru_cache
from typing import Dict, List, Any, Tuple, Optional, Set
import numpy as np
import warnings
import math
from collections import Counter # Required for Cell 8/9, useful here

# NOTE: Dependencies on Grid (Cell 4), dilate_grid/erode_grid/apply_color_map/translate_grid (Cell 5) 
# and other utilities are assumed.

# ---- Core Comparison Functions (V2 Robustness + V3 Caching) ----

@lru_cache(maxsize=1024)
def grids_are_equal(grid1: Grid, grid2: Grid, ignore_background: bool = True) -> bool:
    """Checks if two Grids are identical."""
    if grid1.shape != grid2.shape: return False
    if grid1.size == 0 and grid2.size == 0: return True
    if not ignore_background:
        return np.array_equal(grid1.data, grid2.data)
    else:
        mask1 = (grid1.data != 0)
        mask2 = (grid2.data != 0)
        if not np.array_equal(mask1, mask2): return False
        if np.any(mask1): 
            return np.array_equal(grid1.data[mask1], grid2.data[mask2])
        return True

@lru_cache(maxsize=1024)
def grids_are_structurally_identical(grid1: Grid, grid2: Grid, allow_color_swap: bool = False) -> bool:
    """Checks if two grids have the same non-background pixel locations."""
    if grid1.shape != grid2.shape: return False
    mask1 = (grid1.data != 0)
    mask2 = (grid2.data != 0)
    if not np.array_equal(mask1, mask2): return False
    if allow_color_swap: return True
    return np.array_equal(grid1.data, grid2.data)

@lru_cache(maxsize=1024)
def find_change_vectors(grid_in: Grid, grid_out: Grid, background_color: int = 0) -> List[Tuple[int, int, int, int, int]]:
    """Calculates pixel changes: (r, c, old_c, new_c, status)."""
    if grid_in.shape != grid_out.shape: return []
    data_in = grid_in.data
    data_out = grid_out.data
    r, c = grid_in.shape
    change_vectors = []
    for row in range(r):
        for col in range(c):
            c_in = int(data_in[row, col])
            c_out = int(data_out[row, col])
            if c_in == c_out: continue
            status = 0
            if c_in == background_color and c_out != background_color: status = 1
            elif c_in != background_color and c_out == background_color: status = 2
            elif c_in != background_color and c_out != background_color: status = 3
            else: continue 
            change_vectors.append((row, col, c_in, c_out, status))
    return change_vectors

# ... (Other comparison functions like objects_are_equivalent, find_difference_mask
#      are assumed to be here from the previous correct response) ...


# ---- Hypothesis and Rule Structures (V3 Formalization) ----

@dataclass(frozen=True)
class AbstractRule:
    """
    V3 Base class for all hypotheses/transformation rules. 
    
    PATCH: All default fields are marked as 'kw_only=True'.
    This solves the 'non-default argument follows default argument' TypeError
    by separating positional args from keyword-only args in the __init__ signature.
    """
    rule_id: str  # Non-default field is fine here.
    
    # --- Keyword-Only Default Fields ---
    confidence: float = field(default=0.0, kw_only=True)
    features: Dict[str, Any] = field(default_factory=dict, kw_only=True)
    parameters: Dict[str, Any] = field(default_factory=dict, kw_only=True)
    
    def apply(self, grid: Grid) -> Optional[Grid]:
        raise NotImplementedError("Subclasses must implement the apply method.")
        
    def __str__(self):
        params_str = ", ".join(f"{k}={v}" for k, v in self.parameters.items())
        return f"{self.rule_id}({params_str}) [C:{self.confidence:.2f}]"

@dataclass(frozen=True)
class TransformationRule(AbstractRule):
    """
    V2/V3 Core: Represents a geometric or morphological transformation.
    
    PATCH: This class now correctly inherits. The generated __init__ will be:
    __init__(self, rule_id: str, transform_type: str, *, 
             confidence: float = 0.0, features: ... = ..., parameters: ... = ...)
    This signature is VALID.
    """
    # 1. Non-default fields
    #    (rule_id is inherited from base, transform_type is new)
    transform_type: str
    
    # 2. Default fields (confidence, features, parameters)
    #    are now automatically and correctly inherited as keyword-only.

    def apply(self, grid: Grid) -> Optional[Grid]:
        """Applies the geometric transformation to the Grid."""
        
        func_map = {
            'rotate': grid.rotate_90,
            'flip_h': grid.flip_h,
            'flip_v': grid.flip_v,
            'crop_to_bbox': grid.crop_to_bbox,
            'dilate': dilate_grid, 
            'erode': erode_grid,   
            'translate': translate_grid 
        }
        
        func = func_map.get(self.transform_type)
        if not func:
            warnings.warn(f"Unknown transformation type: {self.transform_type}", UserWarning)
            return None
        
        if self.transform_type in ['dilate', 'erode', 'translate']:
            return func(grid, **self.parameters)
        else:
            return func(**self.parameters)


@dataclass(frozen=True)
class MappingRule(AbstractRule):
    """
    V2/V3 Core: Represents a color, size, or count mapping rule.
    
    PATCH: This class also inherits correctly now.
    __init__(self, rule_id: str, mapping_type: str, *, confidence: ...)
    """
    # 1. Non-default fields
    mapping_type: str
    
    # 2. Default fields (inherited as keyword-only)

    def apply(self, grid: Grid) -> Optional[Grid]:
        """Applies a color or structural mapping to the Grid."""
        
        if self.mapping_type == 'color_swap':
            color_map = self.parameters.get('color_map', {})
            default_color = self.parameters.get('default_color')
            if not color_map:
                return grid 
            return apply_color_map(grid, color_map, default_color)
            
        elif self.mapping_type == 'recolor_all_non_zero':
            new_color = self.parameters.get('new_color')
            if new_color is None or not (0 <= new_color <= 9):
                 return grid
            new_data = grid.data.copy()
            new_data[new_data != 0] = new_color
            return Grid(new_data)
            
        warnings.warn(f"Unknown mapping type: {self.mapping_type}", UserWarning)
        return None

print("Cell 7 ready: Comparators and Abstract Rule Structures (Definitively Patched with kw_only).")


Cell 7 ready: Comparators and Abstract Rule Structures (Definitively Patched with kw_only).


In [9]:
# Cell 8 — Rule Generation, Scoring, and Validation Engine (v2 Deductive Core + v3 Confidence)

# IMPL 6: Hypothesis Generation and Scoring

# ---- Scoring Utility (V2 Heuristics) ----

def score_rule_match(predicted_grid: Grid, target_grid: Grid) -> float:
    """
    V2 Heuristic: Calculates a confidence score (0.0 to 1.0) for how well a predicted 
    grid matches the target grid. This combines shape, pixel-wise accuracy, and 
    structural similarity.
    """
    if not predicted_grid.is_valid or not target_grid.is_valid:
        return 0.0
        
    # 1. Shape Check
    if predicted_grid.shape != target_grid.shape:
        # Penalize heavily, but don't zero out completely if there is substantial overlap
        # V3 R&D: Calculate shape penalty based on difference in area or dimensions
        r1, c1 = predicted_grid.shape
        r2, c2 = target_grid.shape
        area1 = r1 * c1
        area2 = r2 * c2
        
        # Simple size penalty
        size_diff_ratio = abs(area1 - area2) / max(area1, area2, 1)
        # 0.2 is the max score ceiling if shapes don't match
        score_ceiling = 0.2 * (1 - size_diff_ratio)
        
        # If both are non-empty, use intersection over union (IoU) on difference
        r_overlap = min(r1, r2)
        c_overlap = min(c1, c2)
        
        # Crop both to the overlapping region for pixel comparison
        crop_data1 = predicted_grid.data[:r_overlap, :c_overlap]
        crop_data2 = target_grid.data[:r_overlap, :c_overlap]
        
        # Calculate IoU in the overlapping region
        intersection = np.sum(crop_data1 == crop_data2)
        union = np.sum(crop_data1 != 0) + np.sum(crop_data2 != 0) - np.sum((crop_data1 != 0) & (crop_data2 != 0))
        
        iou = intersection / max(union, 1)
        
        # The score is a combination of overlap match and the penalty for size mismatch
        return max(iou * 0.5, score_ceiling)
        
    # 2. Pixel-wise Accuracy (for same-shape grids)
    total_pixels = predicted_grid.size
    if total_pixels == 0:
        return 1.0 # Perfect match for 0x0 grids
        
    matching_pixels = np.sum(predicted_grid.data == target_grid.data)
    
    # Simple accuracy: 1.0 if perfect, 0.0 if zero match
    pixel_score = matching_pixels / total_pixels
    
    # 3. Structural Accuracy (non-background IoU, V2 robustness)
    mask_pred = predicted_grid.data != 0
    mask_target = target_grid.data != 0
    
    mask_intersection = np.sum(mask_pred & mask_target)
    mask_union = np.sum(mask_pred | mask_target)
    
    structural_iou = mask_intersection / max(mask_union, 1)
    
    # V3 Confidence Combination: Weighted average for final score
    # Pixel match is the most important (50%), Structural (30%), Shape (20% implicit in step 1)
    # Since shape is 100% in this branch, we combine the two pixel scores.
    # We use a blend: high structural IoU boosts the score, but low pixel score pulls it down.
    final_score = (pixel_score * 0.7) + (structural_iou * 0.3)
    
    return min(final_score, 1.0)

# ---- Rule Generator Core (V2/V3 Deduction) ----

@dataclass(frozen=True)
class RuleGenerator:
    """
    V3 Structure: A class encapsulating the logic for generating potential 
    TransformationRules and MappingRules from an input/output example pair.
    """
    example_in: Grid
    example_out: Grid
    
    def generate_all_rules(self) -> List[AbstractRule]:
        """Runs all sub-generators and returns a consolidated list of hypotheses."""
        rules = []
        rules.extend(self._generate_geometric_rules())
        rules.extend(self._generate_mapping_rules())
        rules.extend(self._generate_structural_rules())
        
        # V3 R&D: Apply confidence score to all generated rules using the scoring utility
        for rule in rules:
            try:
                predicted_out = rule.apply(self.example_in)
                if predicted_out and predicted_out.is_valid:
                    # V3 Safety: Use object.__setattr__ to update frozen dataclass field
                    score = score_rule_match(predicted_out, self.example_out)
                    object.__setattr__(rule, 'confidence', score)
            except Exception as e:
                # Rule application failed (e.g., bad parameter), set confidence to 0
                object.__setattr__(rule, 'confidence', 0.0)
                object.__setattr__(rule, 'features', {'error': str(e)})

        # Filter out rules with 0 confidence (failed application or 0 match)
        return [rule for rule in rules if rule.confidence > 0.0]


    # ---- Sub-Generator 1: Geometric Transformations (V2 Core) ----
    def _generate_geometric_rules(self) -> List[TransformationRule]:
        """
        Hypothesizes simple rotation, flip, and combined transformation rules.
        """
        rules = []
        
        # Get all 8 symmetries of the input grid
        symmetries = self.example_in.all_symmetries()
        
        # 1. Direct Symmetric Match: Is the output a simple rotation/flip of the input?
        
        # The key is to check which symmetry matches the output exactly (or structurally)
        for i, sym_grid in enumerate(symmetries):
            if grids_are_equal(sym_grid, self.example_out):
                
                # V3 R&D: Determine *which* transformation was applied to get this symmetry.
                # Since Grid.all_symmetries doesn't currently return the transform, we must 
                # re-derive it (a minor v3 patch/improvement needed, but for now, we estimate).
                
                # Simple estimation based on shape change (only accurate for squares)
                # This is a heuristic derived from the v2 pipeline.
                
                transform_id = f"Sym_{i}"
                if sym_grid.shape == self.example_in.shape:
                    if i == 0: transform_id = 'Identity'
                    elif i == 1: transform_id = 'Rotate_90'
                    elif i == 2: transform_id = 'Rotate_180'
                    # ... and so on for the 8 transforms.
                    
                rules.append(TransformationRule(
                    rule_id=f"T_Geom_{transform_id}",
                    transform_type='Identity', # Placeholder, the confidence will be high
                    parameters={},
                    features={'is_direct_match': True}
                ))
        
        # 2. Crop to BBox Rule: Did the operation just crop the input?
        # Check if output is a subset of input's bounding box
        if self.example_in.size > 0:
            cropped = self.example_in.crop_to_bbox()
            if grids_are_equal(cropped, self.example_out):
                rules.append(TransformationRule(
                    rule_id="T_Crop_BBox_All",
                    transform_type='crop_to_bbox',
                    parameters={},
                    features={'bounding_box_only': True}
                ))
                
        # 3. Size Invariance Rule: Output shape == Input shape
        if self.example_in.shape == self.example_out.shape:
             rules.append(TransformationRule(
                 rule_id="T_Size_Invariance",
                 transform_type='Identity', # Marker rule
                 confidence=0.01, # Low confidence, used for filtering later
                 features={'shape_invariance': True}
             ))
             
        return rules

    # ---- Sub-Generator 2: Color Mapping Rules (V2 Core) ----
    def _generate_mapping_rules(self) -> List[MappingRule]:
        """
        Hypothesizes simple color change (recoloring) rules based on frequency 
        and position.
        """
        rules = []
        
        # 1. Simple Color Swap (Frequency-based, V2 core)
        in_counts = self.example_in.get_color_counts(ignore_color=0)
        out_counts = self.example_out.get_color_counts(ignore_color=0)
        
        # We look for a one-to-one mapping based on frequency/position
        color_map = {}
        in_colors = sorted(in_counts.keys())
        out_colors = sorted(out_counts.keys())

        # If the number of unique colors is the same, try a simple frequency-based map
        if len(in_colors) == len(out_colors):
            # Sort pairs by frequency (V2 Heuristic)
            in_freqs = sorted(in_counts.items(), key=lambda item: item[1])
            out_freqs = sorted(out_counts.items(), key=lambda item: item[1])
            
            is_simple_swap = True
            temp_map = {}
            for (c_in, count_in), (c_out, count_out) in zip(in_freqs, out_freqs):
                if count_in != count_out:
                    is_simple_swap = False # Frequencies don't align
                    break
                temp_map[c_in] = c_out
            
            if is_simple_swap and temp_map:
                rules.append(MappingRule(
                    rule_id="M_ColorSwap_Freq",
                    mapping_type='color_swap',
                    parameters={'color_map': temp_map},
                    features={'is_freq_swap': True}
                ))

        # 2. Single Recolor Rule (V2 robustness)
        # If the output is the input recolored to a single dominant color
        dominant_out_color = self.example_out.get_dominant_color(ignore_color=0)
        
        if dominant_out_color is not None:
             # Check if the structure (non-zero mask) is the same
            if grids_are_structurally_identical(self.example_in, self.example_out, allow_color_swap=True):
                rules.append(MappingRule(
                    rule_id="M_Recolor_Single",
                    mapping_type='recolor_all_non_zero',
                    parameters={'new_color': dominant_out_color},
                    features={'is_recolor_all': True}
                ))
                
        return rules

    # ---- Sub-Generator 3: Structural/Object Rules (V2 Advanced Tech) ----
    def _generate_structural_rules(self) -> List[AbstractRule]:
        """
        Hypothesizes rules based on object properties or structural differences (dilate/erode).
        """
        rules = []
        
        # 1. Morphological Rules (Dilate/Erode)
        # Check if output is a simple dilation of input
        for k in [1]: # Check for 1-pixel dilation
            dilated = dilate_grid(self.example_in, k_size=k)
            if score_rule_match(dilated, self.example_out) > 0.95:
                 rules.append(TransformationRule(
                    rule_id=f"T_Dilate_{k}",
                    transform_type='dilate',
                    parameters={'k_size': k},
                    features={'morphological': 'dilate'}
                ))
            
            eroded = erode_grid(self.example_in, k_size=k)
            if score_rule_match(eroded, self.example_out) > 0.95:
                 rules.append(TransformationRule(
                    rule_id=f"T_Erode_{k}",
                    transform_type='erode',
                    parameters={'k_size': k},
                    features={'morphological': 'erode'}
                ))

        # 2. Object-based Rules (V2 R&D)
        
        in_objects = extract_objects(self.example_in)
        out_objects = extract_objects(self.example_out)
        
        # Check for ONE-TO-ONE object mapping: 
        # Same number of objects, and they are equivalent in structure/color/position
        if len(in_objects) == len(out_objects) and len(in_objects) > 0:
            
            # Simple check for object equivalence (ignoring internal object order)
            in_props = sorted([(obj.shape, obj.color, obj.count) for obj in in_objects])
            out_props = sorted([(obj.shape, obj.color, obj.count) for obj in out_objects])
            
            if in_props == out_props:
                rules.append(AbstractRule(
                    rule_id="A_Object_Invariance",
                    confidence=0.8,
                    features={'object_invariance': True, 'count': len(in_objects)}
                ))
                
        # V3 R&D: Add rule for "Translate object by constant vector"
        # Not fully implemented here as it requires complex matching logic, but the structure is ready.
        
        return rules

# ---- Global Task Solver Orchestration (V2/V3 Meta-Logic) ----

def solve_task_with_rules(task: Task, ctx: DebugContext) -> Optional[Grid]:
    """
    The top-level logic that orchestrates rule generation, validation, and selection.
    V2 Core: Find the single rule that applies to ALL train examples.
    """
    
    # 1. Generate candidate rules for the first training example (In -> Out)
    if not task.train:
        return None
        
    ex1 = task.train[0]
    generator = RuleGenerator(ex1.input, ex1.output)
    candidate_rules = generator.generate_all_rules()
    
    # V3 Debug: Log initial set of candidate rules
    ctx.log_pathway("Rules_Candidate_Ex1", [str(r) for r in candidate_rules])
    
    # Filter rules based on minimal confidence (V2 Heuristic)
    C_MIN_THRESHOLD = 0.5
    filtered_rules = [r for r in candidate_rules if r.confidence >= C_MIN_THRESHOLD]
    
    if not filtered_rules:
        ctx.log_pathway("Rules_Failed", "No rule passed minimum confidence threshold.")
        return None
        
    # 2. Rule Validation: Test remaining rules against ALL other train examples
    # The rule must be perfect for all.
    
    best_rule: Optional[AbstractRule] = None
    
    for rule in filtered_rules:
        is_valid_rule = True
        
        # Skip the first example, as the rule was generated from it
        for i, ex in enumerate(task.train[1:]): 
            if not ctx.check_time():
                ctx.log_pathway("Timeout_Rule_Validation", rule.rule_id)
                return None
                
            predicted_out = rule.apply(ex.input)
            
            if predicted_out is None or not predicted_out.is_valid:
                # Rule failed to apply cleanly
                is_valid_rule = False
                break
                
            # Must be a PERFECT match for ALL training examples (V2 Strictness)
            if not grids_are_equal(predicted_out, ex.output, ignore_background=False):
                is_valid_rule = False
                break
                
        if is_valid_rule:
            # V3 Selection: The first fully validated rule is chosen, highest confidence first
            if best_rule is None or rule.confidence > best_rule.confidence:
                best_rule = rule
                
    if best_rule is None:
        ctx.log_pathway("Rules_Failed", "No rule perfectly validated across all examples.")
        return None
        
    # V3 Debug: Log the winning rule
    ctx.log_pathway("Rules_Winner", str(best_rule), best_rule.confidence)

    # 3. Apply the winning rule to the Test set
    final_predictions: List[Grid] = []
    for test_ex in task.test:
        if not ctx.check_time():
            ctx.log_pathway("Timeout_Test_Application", best_rule.rule_id)
            # Return any partial successful predictions or None if 0
            return final_predictions[0] if final_predictions else None
            
        predicted_out = best_rule.apply(test_ex.input)
        
        if predicted_out and predicted_out.is_valid:
            final_predictions.append(predicted_out)
        else:
            # Prediction failed for one test case, the rule is not robust enough
            ctx.log_pathway("Test_Application_Failure", f"Rule {best_rule.rule_id} failed on a test input.")
            return None 

    # V2/V3 Standard: For ARC, we only submit the first test prediction
    return final_predictions[0] if final_predictions else None

# V3 R&D: Import deque for BFS in previous cells, required here for context
from collections import deque
from dataclasses import field # Already imported in Cell 7, but safe to include for reference

print("Cell 8 ready: Rule Generation, Scoring, and Validation Engine loaded.")


Cell 8 ready: Rule Generation, Scoring, and Validation Engine loaded.


In [10]:
# Cell 9 — Metacognitive and Fuzzy Logic Core (v2 R&D Engine + v3 Composite Rules)

# IMPL 7: Fuzzy Logic and Composite Reasoning

@dataclass(frozen=True)
class CompositeRule(AbstractRule):
    """
    V3 Core: Represents a sequence of simpler rules (AbstractRules).
    The Metacognitive engine's primary output for complex tasks.
    """
    rules: List[AbstractRule] = field(default_factory=list)
    
    def apply(self, grid: Grid) -> Optional[Grid]:
        """Applies the sequence of rules in order."""
        current_grid = grid
        for rule in self.rules:
            if current_grid is None or not current_grid.is_valid:
                # Sequence failed midway
                return None
            
            # Apply the next rule in the sequence
            current_grid = rule.apply(current_grid)
            
        return current_grid

    def __str__(self):
        # Detailed string representation
        rule_list_str = " -> ".join([f"{r.rule_id}(C:{r.confidence:.2f})" for r in self.rules])
        return f"CompositeRule [Steps:{len(self.rules)}] | C:{self.confidence:.2f} | {rule_list_str}"

@dataclass
class FuzzyLogicCore:
    """
    V3/V2 Hybrid: Encapsulates advanced logic for finding non-obvious rules,
    specifically dealing with imperfect matches, color ambiguity, and multi-step processes.
    """
    ctx: DebugContext # V3 requirement: Needs access to the debug context
    
    # ---- 1. Fuzzy Color Mapping (V2 Heuristics) ----

    def posited_fuzzy_mapping(self, input_grid: Grid, output_grid: Grid) -> MappingRule:
        """
        V2 Advanced Feature: Hypothesizes a color mapping (color_map) based on 
        the change vectors, prioritizing common/frequent transformations over strict 
        frequency matching (Cell 8).
        """
        change_vectors = find_change_vectors(input_grid, output_grid)
        
        # 1. Count transitions: { (old_color, new_color): count }
        transition_counts = Counter((v[2], v[3]) for v in change_vectors if v[4] == 3) # Only 'Recolored' status
        
        if not transition_counts:
            # If no recoloring, but the structure is the same, assume it's an identity map
            if grids_are_structurally_identical(input_grid, output_grid, allow_color_swap=True):
                 return MappingRule(rule_id="M_Fuzzy_Identity", mapping_type='color_swap', parameters={'color_map': {}})
                 
            # Fallback to a zero-confidence rule
            return MappingRule(rule_id="M_Fuzzy_Fail", mapping_type='color_swap', confidence=0.0)

        # 2. Build the fuzzy map
        # Goal: For each input color, find the *most frequent* resulting output color.
        
        input_colors = sorted(list(set(t[0] for t in transition_counts.keys())))
        color_map = {}
        features = {}

        for c_in in input_colors:
            # Filter transitions that start with c_in
            transitions = [(c_out, count) for (c_i, c_out), count in transition_counts.items() if c_i == c_in]
            
            if not transitions:
                continue

            # Find the most frequent destination color (c_out)
            best_c_out, max_count = max(transitions, key=lambda item: item[1])
            
            # V2 R&D: Log ambiguity if there's a tie or close secondary transition
            total_transitions_from_c_in = sum(count for _, count in transitions)
            is_ambiguous = len(transitions) > 1 and max_count < 0.75 * total_transitions_from_c_in
            
            if is_ambiguous:
                features[f'ambiguity_in_{c_in}'] = True
                
            color_map[c_in] = best_c_out
            
        # V3 R&D: Calculate confidence based on coverage (how many total transitions were covered)
        covered_count = sum(count for (c_in, c_out), count in transition_counts.items() if color_map.get(c_in) == c_out)
        total_recolored = sum(transition_counts.values())
        
        confidence = covered_count / max(total_recolored, 1) # Coverage ratio
        
        return MappingRule(
            rule_id="M_Fuzzy_Transition",
            mapping_type='color_swap',
            parameters={'color_map': color_map},
            features=features,
            confidence=confidence
        )

    # ---- 2. Difference Analysis-Based Structural Rules (V2 R&D) ----

    def _generate_difference_rules(self, input_grid: Grid, output_grid: Grid) -> List[TransformationRule]:
        """
        Generates highly specific rules by analyzing where and how pixels were created or deleted.
        """
        rules = []
        change_vectors = find_change_vectors(input_grid, output_grid)
        
        if not change_vectors:
            return rules # No change, no difference rule needed

        # A. Rule: Bounding Box Recolor/Fill (Creation/Deletion)
        # Check if ALL changes occur inside the bounding box of the input
        
        r_min, r_max, c_min, c_max = input_grid.get_bbox()
        all_changes_in_bbox = all(r_min <= r < r_max and c_min <= c < c_max for r, c, *rest in change_vectors)

        if all_changes_in_bbox and (r_max - r_min > 0 and c_max - c_min > 0):
            # Check for a specific pattern of change (e.g., filling the whole bbox)
            
            # Total cells in BBOX
            bbox_area = (r_max - r_min) * (c_max - c_min)
            
            # Check for a complete "fill" or "clear" operation within the BBOX
            creation_count = sum(1 for *_, status in change_vectors if status == 1)
            deletion_count = sum(1 for *_, status in change_vectors if status == 2)
            
            if creation_count > 0.8 * bbox_area:
                # Hypothesize: The operation was "Fill BBox with dominant created color"
                
                # Find the most common created color
                created_colors = Counter(c_out for *_, c_out, status in change_vectors if status == 1)
                dominant_created_color = created_colors.most_common(1)[0][0] if created_colors else None
                
                if dominant_created_color is not None:
                    rules.append(TransformationRule(
                        rule_id="T_Fill_BBox",
                        transform_type='BBox_Fill', # New custom type
                        parameters={'r_min': r_min, 'r_max': r_max, 'c_min': c_min, 'c_max': c_max, 'fill_color': dominant_created_color},
                        confidence=0.8
                    ))

        # B. Rule: Boundary Extension/Mirroring (Focus on edge changes)
        # Check if created/deleted pixels are predominantly on the boundary of the input object(s)
        
        # This is a complex heuristic (V2 R&D: omitted for length, but the placeholder shows intent)
        
        return rules

    # ---- 3. Composite Rule Generation (V2 Metacognitive Core) ----

    def generate_composite_rules(self, generator: RuleGenerator, top_k: int = 5) -> List[CompositeRule]:
        """
        V2 Metacognitive: Tries to combine simple rules to achieve a high-confidence match 
        where single rules failed.
        """
        all_simple_rules = generator.generate_all_rules()
        
        # 1. Prune rules that are below a threshold, but keep more than in Cell 8
        C_PRUNE_THRESHOLD = 0.3
        simple_candidates = sorted(
            [r for r in all_simple_rules if r.confidence >= C_PRUNE_THRESHOLD], 
            key=lambda r: r.confidence, 
            reverse=True
        )[:top_k]
        
        if not simple_candidates:
            return []

        composite_rules = []
        
        # 2. Two-Step Composite (V2 Standard Depth)
        
        for rule1 in simple_candidates:
            # Apply the first rule
            intermediate_grid = rule1.apply(generator.example_in)
            
            if intermediate_grid is None or not intermediate_grid.is_valid:
                continue

            # Check if the intermediate grid is a near-perfect match for the output
            if score_rule_match(intermediate_grid, generator.example_out) >= 0.99:
                 # Rule 1 alone is sufficient, promote it to a composite of length 1
                 composite_rules.append(CompositeRule(
                     rule_id=f"Comp_{rule1.rule_id}",
                     rules=[rule1],
                     confidence=rule1.confidence,
                     features={'depth': 1}
                 ))
                 continue

            # Now, generate a second set of rules based on (Intermediate -> Output)
            # This is the "residual" analysis
            generator_residual = RuleGenerator(intermediate_grid, generator.example_out)
            residual_rules = generator_residual.generate_all_rules()
            
            # Prune residual rules based on a higher match threshold
            C_RESIDUAL_THRESHOLD = 0.8
            best_residual_rules = sorted(
                [r for r in residual_rules if r.confidence >= C_RESIDUAL_THRESHOLD],
                key=lambda r: r.confidence, 
                reverse=True
            )[:2] # Only take the top 2 residuals

            for rule2 in best_residual_rules:
                # Calculate the overall confidence score for the composite rule
                # V2 Heuristic: Geometric mean of confidence, slightly penalized for complexity (depth)
                
                # Rule 2 must be different from Rule 1 (unless it's Identity or similar neutral ops)
                if rule1.rule_id == rule2.rule_id and rule1.transform_type != 'Identity':
                     continue

                final_confidence = math.sqrt(rule1.confidence * rule2.confidence) * 0.95 # 5% penalty for 2 steps
                
                new_composite = CompositeRule(
                    rule_id=f"Comp_{rule1.rule_id}_then_{rule2.rule_id}",
                    rules=[rule1, rule2],
                    confidence=final_confidence,
                    features={'depth': 2, 'C1': rule1.confidence, 'C2': rule2.confidence}
                )
                composite_rules.append(new_composite)

        # V3 R&D: Filter composites for uniqueness and return the best
        # Use the string representation of the sequence of transforms as a uniqueness key
        unique_composites = {}
        for comp in composite_rules:
            key = tuple(r.rule_id for r in comp.rules)
            if key not in unique_composites or comp.confidence > unique_composites[key].confidence:
                unique_composites[key] = comp
                
        return sorted(list(unique_composites.values()), key=lambda r: r.confidence, reverse=True)


# ---- Extended Global Task Solver Orchestration (Incorporating Fuzzy/Composite) ----

def solve_task_metacognitive(task: Task, ctx: DebugContext) -> Optional[Grid]:
    """
    V3 Top-level function that runs the full R&D pipeline: 
    1. Perfect Rule Deduction (Cell 8)
    2. Fuzzy/Composite Rule Search (Cell 9)
    """
    
    # --- Step 1: Attempt Perfect Rule Deduction (from Cell 8 logic) ---
    
    # We call the core deduction logic, but we need to modify the implementation
    # to return ALL high-confidence, single-step candidates, not just the best validated one.
    
    # To avoid repeating code, we will slightly modify the logic flow here:
    
    ex1 = task.train[0]
    generator = RuleGenerator(ex1.input, ex1.output)
    
    # Generate all single-step rules (Perfect + Fuzzy/Difference)
    single_rules = generator.generate_all_rules()
    
    # Add fuzzy color mapping (often necessary for complex recolor)
    fuzzy_map_rule = FuzzyLogicCore(ctx).posited_fuzzy_mapping(ex1.input, ex1.output)
    if fuzzy_map_rule.confidence > 0:
        single_rules.append(fuzzy_map_rule)
        
    # Add difference-based structural rules
    diff_rules = FuzzyLogicCore(ctx)._generate_difference_rules(ex1.input, ex1.output)
    single_rules.extend(diff_rules)
    
    # Filter rules based on minimal confidence (C_MIN_THRESHOLD defined in Cell 8)
    C_MIN_THRESHOLD = 0.5 
    filtered_rules = [r for r in single_rules if r.confidence >= C_MIN_THRESHOLD]
    
    # --- Step 2: Generate Composite Rules (Fuzzy Core) ---
    
    flc = FuzzyLogicCore(ctx)
    composite_rules = flc.generate_composite_rules(generator, top_k=5)
    
    # --- Step 3: Consolidate and Validate ---
    
    all_candidate_rules = sorted(
        filtered_rules + composite_rules,
        key=lambda r: r.confidence, 
        reverse=True
    )
    
    if not all_candidate_rules:
        ctx.log_pathway("Rules_Failed_All", "No single or composite rule passed confidence.")
        return None
        
    ctx.log_pathway("Rules_Candidate_Total", [str(r) for r in all_candidate_rules])
    
    # 2. Rule Validation: Test remaining rules against ALL other train examples
    best_rule: Optional[AbstractRule] = None
    
    for rule in all_candidate_rules:
        is_valid_rule = True
        
        for i, ex in enumerate(task.train): 
            if not ctx.check_time():
                ctx.log_pathway("Timeout_Rule_Validation", rule.rule_id)
                return None
                
            predicted_out = rule.apply(ex.input)
            
            # Perfect match required for validation
            if predicted_out is None or not predicted_out.is_valid or \
               not grids_are_equal(predicted_out, ex.output, ignore_background=False):
                is_valid_rule = False
                break
                
        if is_valid_rule:
            # V3 Selection: The first fully validated rule (highest confidence) is chosen
            best_rule = rule
            break # Found the winner!
                
    if best_rule is None:
        ctx.log_pathway("Rules_Failed", "No rule perfectly validated across all examples.")
        return None
        
    # V3 Debug: Log the winning rule
    ctx.log_pathway("Rules_Winner_Meta", str(best_rule), best_rule.confidence)

    # 4. Apply the winning rule to the Test set (Same logic as Cell 8)
    final_predictions: List[Grid] = []
    for test_ex in task.test:
        if not ctx.check_time():
            ctx.log_pathway("Timeout_Test_Application", best_rule.rule_id)
            return final_predictions[0] if final_predictions else None
            
        predicted_out = best_rule.apply(test_ex.input)
        
        if predicted_out and predicted_out.is_valid:
            final_predictions.append(predicted_out)
        else:
            ctx.log_pathway("Test_Application_Failure", f"Rule {best_rule.rule_id} failed on a test input.")
            return None 

    return final_predictions[0] if final_predictions else None

# V3 R&D: Re-import required components that might have been defined in previous cells
# These are needed for the functions/classes defined in this cell
from dataclasses import field
from collections import Counter

print("Cell 9 ready: Metacognitive and Fuzzy Logic Core (Composite Rules, Fuzzy Mapping, Difference Analysis) loaded.")


Cell 9 ready: Metacognitive and Fuzzy Logic Core (Composite Rules, Fuzzy Mapping, Difference Analysis) loaded.


In [11]:
# Cell 10 — Task Abstraction, Solver Orchestration, and Submission (v2/v3 Final Integration)

# IMPL 8: Task Abstraction and Invariance Detection

@dataclass
class TaskInvariants:
    """
    V3 Structure: Stores properties that hold true for all (input, output) pairs in a task.
    """
    task_id: str
    shape_change: Optional[str] = None # e.g., 'size_invariant', '2x_scale', 'crop_only'
    color_map: Optional[Dict[int, int]] = None # The dominant, globally consistent color mapping
    bg_invariant: bool = True # Background color is always 0
    num_objects_change: Optional[int] = None # Constant object count change (e.g., +1, -2)
    
    def __str__(self):
        s = f"Invariants({self.task_id}): "
        parts = []
        if self.shape_change: parts.append(f"Shape:{self.shape_change}")
        if self.color_map is not None: parts.append(f"Map:{len(self.color_map)} entries")
        if self.num_objects_change is not None: parts.append(f"Objs_Delta:{self.num_objects_change}")
        return s + ", ".join(parts)


@dataclass
class TaskAnalyzer:
    """
    V2/V3 Hybrid: Analyzes all training examples to find high-level invariants and properties.
    These invariants can be used by the solver (Cell 9) to prune hypotheses or guide search.
    """
    task: Task
    
    def find_invariants(self) -> TaskInvariants:
        """Runs all invariance checks and compiles the results."""
        
        invariants = TaskInvariants(self.task.task_id)
        
        if not self.task.train:
            return invariants

        # 1. Shape Invariance Analysis (V2 Core Feature)
        invariants.shape_change = self._analyze_shape_invariance()

        # 2. Color Mapping Invariance Analysis (V2 Core Feature)
        invariants.color_map = self._analyze_color_mapping_invariance()
        
        # 3. Object Count Invariance Analysis (V2 R&D Feature)
        invariants.num_objects_change = self._analyze_object_count_invariance()

        # V3 R&D: Could add Centroid change invariance, graph feature delta, etc.
        
        return invariants

    def _analyze_shape_invariance(self) -> Optional[str]:
        """Checks if shape changes consistently across all examples."""
        
        # Tracks shape delta: (R_out - R_in, C_out - C_in)
        shape_deltas = set() 
        # Tracks shape ratio: (R_out / R_in, C_out / C_in)
        shape_ratios = set()
        
        is_size_invariant = True
        is_square_invariant = True
        
        for ex in self.task.train:
            r_in, c_in = ex.input.shape
            r_out, c_out = ex.output.shape
            
            # Check size invariance
            if r_in != r_out or c_in != c_out:
                is_size_invariant = False
                
            # Check square invariance (if all are squares, or all inputs are squares and outputs are too)
            if (r_in != c_in) != (r_out != c_out):
                is_square_invariant = False
            
            # Record deltas
            shape_deltas.add((r_out - r_in, c_out - c_in))
            
            # Record ratios (only if input dimensions are non-zero)
            if r_in > 0 and c_in > 0:
                # Use a simple multiplier (e.g., 2x, 3x)
                r_ratio = r_out / r_in
                c_ratio = c_out / c_in
                # Check for integer/half-integer scaling (V2 Heuristic)
                if abs(r_ratio - round(r_ratio)) < 1e-6 and abs(c_ratio - round(c_ratio)) < 1e-6:
                    shape_ratios.add((round(r_ratio), round(c_ratio)))
        
        if is_size_invariant:
            return 'size_invariant'
        
        # Consistent delta (e.g., always adds 1 row and 1 column)
        if len(shape_deltas) == 1:
            dr, dc = list(shape_deltas)[0]
            if dr == 0 and dc == 0:
                 return 'size_invariant' # Redundant, but safe
            return f"delta_r{dr}_c{dc}"

        # Consistent ratio (e.g., always 2x2 scaling)
        if len(shape_ratios) == 1:
            rr, rc = list(shape_ratios)[0]
            return f"scale_{rr}x{rc}"
            
        return 'variable_shape'

    def _analyze_color_mapping_invariance(self) -> Optional[Dict[int, int]]:
        """
        Checks if the dominant color mapping (ignoring background) is consistent 
        across all training examples.
        """
        consistent_maps: List[Dict[int, int]] = []
        
        for ex in self.task.train:
            # V3 Integration: Use the Fuzzy Logic Core to hypothesize the best map
            flc = FuzzyLogicCore(self.task.task_id) # Task ID is a required arg, but only used for logging here
            fuzzy_rule = flc.posited_fuzzy_mapping(ex.input, ex.output)
            
            if fuzzy_rule.confidence > 0.8: # Only accept high-confidence maps
                consistent_maps.append(fuzzy_rule.parameters.get('color_map', {}))

        if not consistent_maps:
            return None
            
        # Check if all maps are identical
        first_map = consistent_maps[0]
        is_consistent = all(m == first_map for m in consistent_maps)
        
        if is_consistent:
            return first_map
        
        return None # No globally consistent, high-confidence map found

    def _analyze_object_count_invariance(self) -> Optional[int]:
        """Checks if the number of objects changes consistently."""
        
        object_deltas = set() # (Objects_out - Objects_in)
        
        for ex in self.task.train:
            num_in = len(extract_objects(ex.input))
            num_out = len(extract_objects(ex.output))
            object_deltas.add(num_out - num_in)
            
        if len(object_deltas) == 1:
            return list(object_deltas)[0]
            
        return None


# IMPL 9: Solver Orchestration (The Main Loop)

def run_solver(task_ids: List[str], max_tasks: Optional[int] = None) -> Dict[str, np.ndarray]:
    """
    V2/V3 Core: Main loop for task processing, respecting the global time budget.
    Returns a dictionary of {task_id: predicted_output_grid}.
    """
    
    # Global metrics instance is assumed to be defined in Cell 1
    global metrics 
    start_time = time.time()
    
    submission_results: Dict[str, np.ndarray] = {}
    
    if max_tasks is not None:
        task_ids = task_ids[:max_tasks]
        
    metrics.task_count = len(task_ids) # Update total task count
    
    print(f"\n--- 🏁 Starting Solver Run on {len(task_ids)} Tasks ---")
    
    for i, task_id in enumerate(task_ids):
        current_global_time = time.time() - start_time
        
        # Check total time budget (V3 Safety)
        if current_global_time > TOTAL_TIME_BUDGET:
            print(f"🛑 Global time budget exceeded ({TOTAL_TIME_BUDGET:.2f}s). Halting solver.")
            metrics.log_metric("global_timeout", {"task": task_id, "index": i})
            break
            
        task_start_time = time.time()
        
        # Load Task (Cell 3)
        task = load_task(task_id)
        if task is None:
            metrics.log_failure(task_id, "Load_Fail", time.time() - task_start_time)
            continue
            
        # V3 Metacognition: Analyze the task *before* solving
        analyzer = TaskAnalyzer(task)
        invariants = analyzer.find_invariants()
        metrics.log_metric(f"invariants_{task_id}", str(invariants))
        
        # Initialize Debug Context (V3 Safety)
        ctx = DebugContext(time_limit_s=TIME_LIMIT_PER_TASK)
        
        # ------------------------------------------------
        # V3 Solver Call (Cell 9)
        predicted_grid = solve_task_metacognitive(task, ctx)
        # ------------------------------------------------
        
        time_taken = time.time() - task_start_time
        
        if predicted_grid and predicted_grid.is_valid:
            # We assume a successful solution has 1.0 confidence for now, 
            # as solve_task_metacognitive only returns a grid if a perfect rule was found.
            confidence = 1.0 
            metrics.log_success(task_id, best_rule.rule_id if 'best_rule' in locals() else "Unknown", confidence, time_taken)
            submission_results[task_id] = predicted_grid.data
            print(f"✅ Solved: {task_id} in {time_taken:.2f}s | Rule: {ctx.pathways[-1][1] if ctx.pathways else 'N/A'}")
        else:
            # Solution failed or timed out
            fail_reason = ctx.pathways[-1][0] if ctx.pathways else "Timeout_NoPath"
            metrics.log_failure(task_id, fail_reason, time_taken)
            print(f"❌ Failed: {task_id} in {time_taken:.2f}s | Reason: {fail_reason}")
            
        metrics.records.append({"task_id": task_id, "dt": time_taken, "success": (predicted_grid is not None)})
        
    return submission_results

# IMPL 10: Submission Finalization (V2 Requirement)

def finalize_submission_from_results(
    submission_results: Dict[str, np.ndarray], 
    all_task_ids: List[str], 
    out_csv_path: Optional[str] = '/kaggle/working/submission.csv'
) -> List[Tuple[str, str]]:
    """
    V2 Core: Converts the dictionary of predicted grids into the ARC submission list 
    format and optionally saves it as a CSV.
    """
    submission: List[Tuple[str, str]] = []
    
    # Use V2's robust flattener (from Cell 2)
    
    for task_id in all_task_ids:
        predicted_grid_data = submission_results.get(task_id)
        
        if predicted_grid_data is not None:
            # Convert numpy array back into a Grid object for the flattener
            predicted_grid = Grid(predicted_grid_data) 
            flat_pred = flattener(predicted_grid.data) # flattener expects np.ndarray
        else:
            # Default submission for failed/unsolved tasks is an empty string
            flat_pred = ""
            
        submission.append((task_id, flat_pred))
        
    # V3 Safety: If running in Kaggle environment, write the CSV
    if out_csv_path is not None:
        try:
            import pandas as pd
            df = pd.DataFrame(submission, columns=['output_id', 'output'])
            df.to_csv(out_csv_path, index=False)
            print(f"\n💾 Submission file saved to: {out_csv_path}")
        except ImportError:
            warnings.warn("Pandas not available. Skipping CSV file creation.", ImportWarning)

    return submission

# V3 R&D: Re-import required components
from dataclasses import field

# V3 R&D: Required imports from previous cells for successful execution
from typing import Set
from collections import deque

print("Cell 10 ready: Task Abstraction (Invariants), Solver Orchestration (run_solver), and Submission Finalization loaded.")


Cell 10 ready: Task Abstraction (Invariants), Solver Orchestration (run_solver), and Submission Finalization loaded.


In [12]:
# Cell 11 — Epistemic Confidence, Memory Consolidation, and Final Reporting (v2/v3 Metacognition)

# IMPL 11: System Self-Reporting and Epistemic Core

# ---- V3 Core Structures (Required from Cell 1 for self-containment) ----

@dataclass
class DebugContext:
    """
    V3 Core: Tracks context-specific data and enforces time limits per task.
    """
    time_limit_s: float
    start_time: float = field(default_factory=time.time)
    pathways: List[Tuple[str, Any, Optional[float]]] = field(default_factory=list) # (event_type, message, confidence)

    def log_pathway(self, event_type: str, message: Any, confidence: Optional[float] = None):
        """Logs a key decision path or finding."""
        self.pathways.append((event_type, message, confidence))

    def check_time(self) -> bool:
        """Returns True if time remaining, False if time limit exceeded."""
        return (time.time() - self.start_time) < self.time_limit_s

# NOTE: The actual global 'metrics' instance must be initialized in Cell 1/2.
# We define the structure here for reference.
@dataclass
class MetricsTracker:
    """
    V3 Core: Global tracker for solver performance and advanced system metrics.
    """
    records: List[Dict[str, Any]] = field(default_factory=list)
    task_count: int = 0
    start_time: float = field(default_factory=time.time)
    
    def log_metric(self, key: str, value: Any):
        """Logs a custom metric for final reporting."""
        # For simplicity, we just print the logging (in a real system, this would write to a log file)
        # print(f"METRIC: {key} = {value}")
        pass # In a unified environment, this would update a global dictionary

    def log_success(self, task_id: str, rule_id: str, confidence: float, time_taken: float):
        self.records.append({"task_id": task_id, "success": True, "rule_id": rule_id, "confidence": confidence, "dt": time_taken})

    def log_failure(self, task_id: str, reason: str, time_taken: float):
        self.records.append({"task_id": task_id, "success": False, "reason": reason, "dt": time_taken})

    def save_metrics(self):
        # Placeholder for saving metrics to a file (V2 requirement)
        # print(f"Metrics saved: {len(self.records)} total entries.")
        pass

# --- Epistemic and Memory Core Functions (V3 R&D) ---

def calculate_epistemic_confidence(metrics: MetricsTracker) -> Dict[str, float]:
    """
    V3 R&D Feature: Calculates the system's *certainty* in its solutions.
    
    Epistemic Confidence: How certain is the system that its solutions are correct?
    - Based on the quality and stability of the winning rules.
    - Uses logged confidence scores and complexity of the final rule.
    """
    
    successful_records = [r for r in metrics.records if r.get('success')]
    
    if not successful_records:
        return {"epistemic_confidence": 0.0, "avg_confidence": 0.0, "composite_ratio": 0.0}
        
    total_confidence = sum(r.get('confidence', 0.0) for r in successful_records)
    avg_confidence = total_confidence / len(successful_records)
    
    # V2/V3 Heuristic: Penalize composite rules, as they are less robust.
    # We assume 'Comp' in the rule_id indicates a CompositeRule (from Cell 9).
    composite_count = sum(1 for r in successful_records if 'Comp' in r.get('rule_id', ''))
    composite_ratio = composite_count / len(successful_records)
    
    # Final confidence: Weighted average, penalized by complexity (composite rules)
    epistemic_confidence = avg_confidence * (1.0 - 0.2 * composite_ratio) # Max 20% penalty
    epistemic_confidence = max(0.0, min(1.0, epistemic_confidence))
    
    return {
        "epistemic_confidence": epistemic_confidence,
        "avg_confidence": avg_confidence,
        "composite_ratio": composite_ratio
    }


def consolidate_memory_health(metrics: MetricsTracker, final_confidence: float) -> Dict[str, Any]:
    """
    V3 R&D Feature: Simulates memory optimization based on solver efficiency.
    
    Memory Quality: How efficient was the search for the final rule?
    - If many rules were generated but few validated, memory health is low (inefficient search).
    - If successful rules were found quickly (low dt), memory quality is high.
    """
    
    # 1. Successful Rule Efficiency (V2 Time Heuristic)
    successful_records = [r for r in metrics.records if r.get('success')]
    avg_dt_success = sum(r.get('dt', 0) for r in successful_records) / max(len(successful_records), 1)
    
    # Normalize time based on the general expected time limit (V3 safety)
    TIME_NORM_FACTOR = TIME_LIMIT_PER_TASK # Assuming max 2.0s per task is ideal
    time_efficiency = max(0.0, min(1.0, 1.0 - (avg_dt_success / TIME_NORM_FACTOR)))

    # 2. General Performance and Memory Optimization
    
    tasks_solved = len(successful_records)
    total_tasks = metrics.task_count
    
    # V3/V2 Heuristic: Quality is a blend of time efficiency and success rate, boosted by high confidence
    memory_quality = (time_efficiency * 0.4) + \
                     ((tasks_solved / max(total_tasks, 1)) * 0.4) + \
                     (final_confidence * 0.2)

    memory_optimized = tasks_solved > 0 and memory_quality > 0.75
    
    return {
        "memory_quality": max(0.0, min(1.0, memory_quality)),
        "memory_optimized": memory_optimized
    }


def finalize_solver_run(metrics: MetricsTracker):
    """
    V2/V3 Final: Executes the self-reporting core and outputs the final summary.
    """
    total_tasks = metrics.task_count
    tasks_solved = len([r for r in metrics.records if r.get('success')])
    total_time = time.time() - metrics.start_time

    # 1. Epistemic Confidence Calculation
    epistemic_results = calculate_epistemic_confidence(metrics)
    final_confidence = epistemic_results['epistemic_confidence']
    
    # 2. Memory Consolidation
    final_consolidation = consolidate_memory_health(metrics, final_confidence)
    
    # V3 Insight 1: Log the final metacognitive steps
    metrics.log_metric("epistemic_results", epistemic_results)
    metrics.log_metric("consolidation_results", final_consolidation)

    print("\n" + "=" * 100)
    print("🧠 Metacognitive Analysis Complete")
    print(f"✅ Tasks Solved: {tasks_solved}/{total_tasks} | Success Rate: {(tasks_solved / total_tasks * 100 if total_tasks > 0 else 0):.2f}%")
    print(f"📊 Total Runtime: {total_time:.2f}s")
    print(f"🌟 System Epistemic Confidence: {epistemic_results['epistemic_confidence']:.2f}")
    
    # V3 Insight 2: Render performance summary table using markdown for Kaggle output
    performance_summary = {
        "Total Tasks": total_tasks,
        "Solved": tasks_solved,
        "Success Rate": f"{(tasks_solved / total_tasks * 100 if total_tasks > 0 else 0):.2f}%",
        "Total Time": f"{total_time:.2f}s",
        "Memory Health (Quality)": f"{final_consolidation['memory_quality']:.2f}",
        "Epistemic Confidence": f"{final_confidence:.2f}",
        "Composite Rule Usage": f"{epistemic_results['composite_ratio']:.2f}"
    }

    print("\n📊 Performance Summary Table:")
    # V2/V3 Output requirement: Use Markdown table format
    print("| Metric | Value |")
    print("|--------|-------|")
    for key, value in performance_summary.items():
        print(f"| {key} | {value} |")

    # Save final metrics with enhanced details (V2 requirement)
    metrics.log_metric("final_summary", performance_summary)
    metrics.save_metrics()

    print("\n🎯 System Wrap-Up Complete. Ready for Evaluation.")
    print("=" * 100)

# V3 R&D: Required imports from previous cells
from dataclasses import field

# V2/V3 FINAL EXECUTION:
# This part orchestrates the run, assuming global variables like 
# ALL_TASK_IDS and the global 'metrics' instance are initialized in Cell 1.

# Example placeholder for global 'metrics' instance initialization
# if 'metrics' not in globals():
#     metrics = MetricsTracker()
#     ALL_TASK_IDS = get_all_task_ids() # Assumed from Cell 3

# V3 EXECUTION FLOW (Commented out, as this is a code cell output):

# submission_results = run_solver(ALL_TASK_IDS, max_tasks=100)
# submission = finalize_submission_from_results(submission_results, ALL_TASK_IDS)
# finalize_solver_run(metrics)

print("Cell 11 ready: Epistemic Confidence, Memory Consolidation, and Final Reporting Core loaded.")


Cell 11 ready: Epistemic Confidence, Memory Consolidation, and Final Reporting Core loaded.


In [13]:
import json
import numpy as np
import os
from pathlib import Path
from typing import Dict, Any, List, Optional

# --- Configuration and Path Setup (CORRECTED FILENAME) ---
# The input files are mounted to the standard Kaggle competition directory
INPUT_DIR = Path("/kaggle/input/arc-prize-2025/")
# FIX: Changed 'arc-agi_test-challenges.json' to 'arc-agi_test_challenges.json'
TEST_CHALLENGES_FILE = INPUT_DIR / "arc-agi_test_challenges.json" 
SUBMISSION_FILE = Path("/kaggle/working/submission.json")

print(f"Loading challenges from: {TEST_CHALLENGES_FILE}")

# --- Helper Function to Load Data ---
def load_json_data(file_path: Path) -> Dict[str, Any]:
    """Loads a JSON file from the given path."""
    try:
        # Check if file exists before trying to open
        if not file_path.exists():
            print(f"Error: File not found at {file_path}. Did you mean to use evaluation challenges?")
            # For robustness, we will try to load the evaluation challenges if test file is not found
            if 'test_challenges' in str(file_path):
                eval_path = INPUT_DIR / "arc-agi_evaluation_challenges.json"
                print(f"Attempting to load evaluation challenges instead: {eval_path}")
                if eval_path.exists():
                    file_path = eval_path
                else:
                    return {}
            else:
                return {}

        with open(file_path, 'r') as f:
            data = json.load(f)
        return data
    except Exception as e:
        print(f"Error loading JSON file {file_path}: {e}")
        return {}

# --- Placeholder/Adapter for Your Solver Logic (Same as before) ---
# NOTE: You MUST replace the content of this function with a call to your actual V3 Solver logic.
def solve_arc_task_and_format(task: Dict[str, Any]) -> List[Dict[str, Any]]:
    """
    Adapter function that calls your core solver logic for a single task
    and formats the result into the required submission structure.
    """
    results_for_task = []
    
    # Placeholder for a solver function that might be defined elsewhere in your notebook
    # Replace this with your actual prediction logic!
    def make_placeholder_prediction(input_grid):
        """Generates a 1x1 black grid as a stand-in prediction."""
        if not input_grid:
             return [[0]]
        # In a real ARC submission, you must predict the *correct* size and content.
        # This placeholder just returns a 1x1 grid for all attempts.
        return [[0]] 
        
    # Iterate over all test inputs for the current task
    for test_pair in task.get("test", []):
        test_input_grid = test_pair["input"]
        
        # ----------------------------------------------------------------------
        # 🎯 CRITICAL: INTEGRATE YOUR SOLVER HERE 🎯
        # Replace the placeholder grid generation with your actual solver call.
        # ----------------------------------------------------------------------
        
        # Example: Replace these two lines with calls to your RhodiumOrcaV3 solver
        attempt_1_grid = make_placeholder_prediction(test_input_grid)
        attempt_2_grid = make_placeholder_prediction(test_input_grid)
        
        results_for_task.append({
            "attempt_1": attempt_1_grid,
            "attempt_2": attempt_2_grid,
        })
        
    return results_for_task


# --- Main Execution Flow ---
def generate_submission_file():
    """Loads test data, runs the solver on all tasks, and saves the submission."""
    
    # 1. Load Test Challenges
    test_challenges = load_json_data(TEST_CHALLENGES_FILE)
    if not test_challenges:
        print("🛑 Could not load test challenges. Aborting submission generation.")
        return

    print(f"✅ Successfully loaded {len(test_challenges)} tasks from the challenges file.")
    
    submission_data = {}
    total_tasks = len(test_challenges)
    
    print("--- 🏁 Starting Solver Run on Test Challenges ---")
    
    # 2. Iterate and Solve Tasks
    for i, (task_id, task_data) in enumerate(test_challenges.items()):
        
        # Call the adapter function to get formatted predictions
        predictions = solve_arc_task_and_format(task_data)
        
        # Store the result in the submission dictionary
        submission_data[task_id] = predictions

    # 3. Save Submission
    print(f"--- ✅ Solver Run Complete. Writing {len(submission_data)} predictions. ---")

    try:
        # Ensure the output directory exists
        os.makedirs(SUBMISSION_FILE.parent, exist_ok=True)
        
        with open(SUBMISSION_FILE, 'w') as f:
            json.dump(submission_data, f)
            
        print(f"💾 Submission file saved to: {SUBMISSION_FILE}")
        
    except Exception as e:
        print(f"🛑 Error saving submission file: {e}")

# Execute the main function
generate_submission_file()

print("\n🎯 System Wrap-Up Complete. Submission ready for evaluation.")


Loading challenges from: /kaggle/input/arc-prize-2025/arc-agi_test_challenges.json
✅ Successfully loaded 240 tasks from the challenges file.
--- 🏁 Starting Solver Run on Test Challenges ---
--- ✅ Solver Run Complete. Writing 240 predictions. ---
💾 Submission file saved to: /kaggle/working/submission.json

🎯 System Wrap-Up Complete. Submission ready for evaluation.
